# Code From Docstrings

- This notebook contains all of the examples found in the documentation (https://veroviz.org/docs).
- This notebook was last tested with VeRoViz version 0.2.0.

--- 

In [1]:
import veroviz as vrv

vrv.checkVersion()

'Your current installed version of veroviz is 0.2.3, the latest version available is 0.2.2. To update to the latest version, type `pip install --upgrade veroviz` at a command-line prompt.'

In [2]:
import os

---
## createAssignments Module

This module includes the following functions:
- `addAssignment2D()`
- `addAssignment3D()`
- `addStaticAssignment()`
- `createAssignmentsFromArcs2D()`
- `createAssignmentsFromLocSeq2D()`
- `createAssignmentsFromNodeSeq2D()`

### `addAssignment2D()`
This function appends to an existing `Assignments` dataframe, or creates a new `Assignments` dataframe if `initAssignments` is `None`.  The new rows in this dataframe describe all of the "shapepoints" between between given starting and ending locations, including timestamps indicating the departure and arrival times for each shapepoint. Shapepoints are pairs of GPS coordinates that are connected by straight lines.  For a given origin and destination, numerous individual shapepoints can be combined to define a travel route along a road network.  

- See https://veroviz.org/docs/veroviz.createAssignments.html#veroviz.createAssignments.addAssignment2D for full documentation of this function.

In [1]:
# Import veroviz and check the latest version.
import veroviz as vrv

vrv.checkVersion()

'Your current installed version of veroviz is 0.2.3, the latest version available is 0.2.2. To update to the latest version, type `pip install --upgrade veroviz` at a command-line prompt.'

In [2]:
# Define 5 node locations, as [lat, lon] pairs:
locs = [[42.8871085, -78.8731949],
        [42.8888311, -78.8649649],
        [42.8802158, -78.8660787],
        [42.8845705, -78.8762794],
        [42.8908031, -78.8770140]]

In [3]:
# Generate a nodes dataframe from these locations:
myNodes = vrv.createNodesFromLocs(locs=locs)
myNodes

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,1,42.887108,-78.873195,0,None,None,glyphicon,info-sign,blue,1,pin,Cesium.Color.BLUE,1
1,2,42.888831,-78.864965,0,None,None,glyphicon,info-sign,blue,2,pin,Cesium.Color.BLUE,2
2,3,42.880216,-78.866079,0,None,None,glyphicon,info-sign,blue,3,pin,Cesium.Color.BLUE,3
3,4,42.884571,-78.876279,0,None,None,glyphicon,info-sign,blue,4,pin,Cesium.Color.BLUE,4
4,5,42.890803,-78.877014,0,None,None,glyphicon,info-sign,blue,5,pin,Cesium.Color.BLUE,5


In [4]:
# View these nodes on a map:
vrv.createLeaflet(nodes=myNodes)

In [5]:
# We're going to hard-code a solution.
# A car will start at node 1, visit nodes 2 and 3, and then return to node 1.
# A truck will follow a route from 1->5->4->1.
mySolution = {
    'car': [[1,2], [2,3], [3,1]],
    'truck': [[1,5], [5,4], [4,1]]
}

In [6]:
# Define some information about our 2 vehicles, for later use:
vehicleProperties = {
    'car':   {'model': 'veroviz/models/car_red.gltf',
              'leafletColor': 'red',
              'cesiumColor': 'Cesium.Color.RED'},
    'truck': {'model': 'veroviz/models/ub_truck.gltf',
              'leafletColor': 'blue',
              'cesiumColor': 'Cesium.Color.BLUE'}
}

In [7]:
# The following examples assume the use of ORS as the data provider. 
# If you have saved your API key as an environment variable, you may use `os.environ` to access it:
import os
ORS_API_KEY = os.environ['ORSKEY']
# Otherwise, you may specify your key here:
# ORS_API_KEY = 'YOUR_ORS_KEY_GOES_HERE'

Example 1 -- The vehicles will visit the nodes in their routes, without any service times.  Assume Euclidean travel (ignoring the road network).

In [8]:
# Build the assignments dataframe for the 2 vehicle routes.
# No service times, Euclidean travel:
myAssignments = vrv.initDataframe('assignments')
for v in mySolution:
    endTimeSec = 0.0
    for arc in mySolution[v]:
        [myAssignments, endTimeSec] = vrv.addAssignment2D(
            initAssignments = myAssignments,
            objectID        = v,
            modelFile       = vehicleProperties[v]['model'],
            startLoc        = list(myNodes[myNodes['id'] == arc[0]][['lat', 'lon']].values[0]),
            endLoc          = list(myNodes[myNodes['id'] == arc[1]][['lat', 'lon']].values[0]),
            startTimeSec    = endTimeSec,
            routeType       = 'euclidean2D',
            speedMPS        = vrv.convertSpeed(25, 'miles', 'hour', 'meters', 'second'),
            leafletColor    = vehicleProperties[v]['leafletColor'],
            cesiumColor     = vehicleProperties[v]['cesiumColor'])
myAssignments

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,car,/veroviz/models/car_red.gltf,100,75,0.000000,42.887108,-78.873195,0,62.545331,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
1,2,car,/veroviz/models/car_red.gltf,100,75,62.545331,42.888831,-78.864965,0,148.568576,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
2,3,car,/veroviz/models/car_red.gltf,100,75,148.568576,42.880216,-78.866079,0,234.592456,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
3,4,truck,/veroviz/models/ub_truck.gltf,100,75,0.000000,42.887108,-78.873195,0,46.129575,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
4,5,truck,/veroviz/models/ub_truck.gltf,100,75,46.129575,42.890803,-78.877014,0,108.314642,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
5,6,truck,veroviz/models/ub_truck.gltf,100,75,108.314642,42.884571,-78.876279,0,142.149457,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8


In [9]:
# Show the routes (and nodes) on a map:
vrv.createLeaflet(nodes=myNodes, arcs=myAssignments)

Example 2 -- The vehicles will now travel on the road network, but we'll still ignore service times.

In [10]:
# No service times, Travel on road network:
myAssignments = vrv.initDataframe('assignments')
for v in mySolution:
    endTimeSec = 0.0
    for arc in mySolution[v]:
        [myAssignments, endTimeSec] = vrv.addAssignment2D(
            initAssignments  = myAssignments,
            objectID         = v,
            modelFile        = vehicleProperties[v]['model'],
            startLoc         = list(myNodes[myNodes['id'] == arc[0]][['lat', 'lon']].values[0]),
            endLoc           = list(myNodes[myNodes['id'] == arc[1]][['lat', 'lon']].values[0]),
            startTimeSec     = endTimeSec,
            leafletColor     = vehicleProperties[v]['leafletColor'],
            cesiumColor      = vehicleProperties[v]['cesiumColor'], 
            routeType        = 'fastest',
            dataProvider     = 'ORS-online', 
            dataProviderArgs = {'APIkey': ORS_API_KEY})
myAssignments        

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,car,/veroviz/models/car_red.gltf,100,75,0.000000,42.887083,-78.873203,0,4.549189,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
1,1,car,/veroviz/models/car_red.gltf,100,75,4.549189,42.887044,-78.872978,0,6.800000,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
2,1,car,/veroviz/models/car_red.gltf,100,75,6.800000,42.887031,-78.872865,0,7.466153,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
3,1,car,/veroviz/models/car_red.gltf,100,75,7.466153,42.887075,-78.872851,0,11.430232,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
4,1,car,/veroviz/models/car_red.gltf,100,75,11.430232,42.887336,-78.872763,0,27.616986,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
5,1,car,/veroviz/models/car_red.gltf,100,75,27.616986,42.888402,-78.872405,0,28.300000,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
6,1,car,/veroviz/models/car_red.gltf,100,75,28.300000,42.888447,-78.872390,0,28.945037,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
7,1,car,/veroviz/models/car_red.gltf,100,75,28.945037,42.888434,-78.872315,0,37.579881,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
8,1,car,/veroviz/models/car_red.gltf,100,75,37.579881,42.888260,-78.871311,0,38.645072,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
9,1,car,/veroviz/models/car_red.gltf,100,75,38.645072,42.888239,-78.871187,0,39.444608,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8


In [11]:
# Show the routes (and nodes) on a map:
vrv.createLeaflet(nodes=myNodes, arcs=myAssignments)

Example 3 -- The vehicles are still following the road network, but now we'll force them to match the travel times specified in a travel matrix.

In [12]:
# We'll first create the travel time and distance matrices:
[timeSec, distMeters] = vrv.getTimeDist2D(nodes            = myNodes,
                                          routeType        = 'fastest',
                                          dataProvider     = 'ORS-online',
                                          dataProviderArgs = {'APIkey': ORS_API_KEY})

In [13]:
# No service times, Travel on road network, use travel times from the distance matrix:
# added "expDurationSec"
myAssignments = vrv.initDataframe('assignments')
for v in mySolution:
    endTimeSec = 0.0
    for arc in mySolution[v]:
        [myAssignments, endTimeSec] = vrv.addAssignment2D(
            initAssignments  = myAssignments,
            objectID         = v,
            modelFile        = vehicleProperties[v]['model'],
            startLoc         = list(myNodes[myNodes['id'] == arc[0]][['lat', 'lon']].values[0]),
            endLoc           = list(myNodes[myNodes['id'] == arc[1]][['lat', 'lon']].values[0]),
            startTimeSec     = endTimeSec,
            expDurationSec   = timeSec[arc[0], arc[1]],
            leafletColor     = vehicleProperties[v]['leafletColor'],
            cesiumColor      = vehicleProperties[v]['cesiumColor'], 
            routeType        = 'fastest',
            dataProvider     = 'ORS-online', 
            dataProviderArgs = {'APIkey': ORS_API_KEY})
myAssignments   

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,car,/veroviz/models/car_red.gltf,100,75,0.000000,42.887083,-78.873203,0,1.889538,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
1,1,car,/veroviz/models/car_red.gltf,100,75,1.889538,42.887044,-78.872978,0,2.824429,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
2,1,car,/veroviz/models/car_red.gltf,100,75,2.824429,42.887031,-78.872865,0,3.326730,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
3,1,car,/veroviz/models/car_red.gltf,100,75,3.326730,42.887075,-78.872851,0,6.315780,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
4,1,car,/veroviz/models/car_red.gltf,100,75,6.315780,42.887336,-78.872763,0,18.521138,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
5,1,car,/veroviz/models/car_red.gltf,100,75,18.521138,42.888402,-78.872405,0,19.036154,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
6,1,car,/veroviz/models/car_red.gltf,100,75,19.036154,42.888447,-78.872390,0,19.665987,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
7,1,car,/veroviz/models/car_red.gltf,100,75,19.665987,42.888434,-78.872315,0,28.097291,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
8,1,car,/veroviz/models/car_red.gltf,100,75,28.097291,42.888260,-78.871311,0,29.137373,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
9,1,car,/veroviz/models/car_red.gltf,100,75,29.137373,42.888239,-78.871187,0,29.918064,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8


Example 4 -- Add service times at each destination node

In [14]:
# 60-second service times at destinations, Travel on road network, use travel times from the distance matrix.
# Added use of `addStaticAssignment()` function for the service component.
myAssignments = vrv.initDataframe('assignments')
for v in mySolution:
    endTimeSec = 0.0
    for arc in mySolution[v]:
        [myAssignments, endTimeSec] = vrv.addAssignment2D(
            initAssignments  = myAssignments,
            objectID         = v,
            modelFile        = vehicleProperties[v]['model'],
            startLoc         = list(myNodes[myNodes['id'] == arc[0]][['lat', 'lon']].values[0]),
            endLoc           = list(myNodes[myNodes['id'] == arc[1]][['lat', 'lon']].values[0]),
            startTimeSec     = endTimeSec,
            expDurationSec   = timeSec[arc[0], arc[1]],
            leafletColor     = vehicleProperties[v]['leafletColor'],
            cesiumColor      = vehicleProperties[v]['cesiumColor'], 
            routeType        = 'fastest',
            dataProvider     = 'ORS-online', 
            dataProviderArgs = {'APIkey': ORS_API_KEY})
        
        myAssignments = vrv.addStaticAssignment(
            initAssignments  = myAssignments,
            objectID         = v,
            modelFile        = vehicleProperties[v]['model'],
            loc              = list(myNodes[myNodes['id'] == arc[1]][['lat', 'lon']].values[0]),
            startTimeSec     = endTimeSec,
            endTimeSec       = endTimeSec + 60.0)
        
        endTimeSec += 60
myAssignments     

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,car,/veroviz/models/car_red.gltf,100,75,0.000000,42.887083,-78.873203,0,1.889538,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
1,1,car,/veroviz/models/car_red.gltf,100,75,1.889538,42.887044,-78.872978,0,2.824429,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
2,1,car,/veroviz/models/car_red.gltf,100,75,2.824429,42.887031,-78.872865,0,3.326730,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
3,1,car,/veroviz/models/car_red.gltf,100,75,3.326730,42.887075,-78.872851,0,6.315780,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
4,1,car,/veroviz/models/car_red.gltf,100,75,6.315780,42.887336,-78.872763,0,18.521138,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
5,1,car,/veroviz/models/car_red.gltf,100,75,18.521138,42.888402,-78.872405,0,19.036154,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
6,1,car,/veroviz/models/car_red.gltf,100,75,19.036154,42.888447,-78.872390,0,19.665987,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
7,1,car,/veroviz/models/car_red.gltf,100,75,19.665987,42.888434,-78.872315,0,28.097291,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
8,1,car,/veroviz/models/car_red.gltf,100,75,28.097291,42.888260,-78.871311,0,29.137373,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
9,1,car,/veroviz/models/car_red.gltf,100,75,29.137373,42.888239,-78.871187,0,29.918064,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8


Example 5 -- Extend the previous example to show packages being left at destination nodes

In [15]:
# 30-second service times at destinations, Travel on road network:
# added another use of `addStaticAssignment()` function to drop packages.
myAssignments = vrv.initDataframe('assignments')
for v in mySolution:
    endTimeSec = 0.0
    for arc in mySolution[v]:
        [myAssignments, endTimeSec] = vrv.addAssignment2D(
            initAssignments  = myAssignments,
            objectID         = v,
            modelFile        = vehicleProperties[v]['model'],
            startLoc         = list(myNodes[myNodes['id'] == arc[0]][['lat', 'lon']].values[0]),
            endLoc           = list(myNodes[myNodes['id'] == arc[1]][['lat', 'lon']].values[0]),
            startTimeSec     = endTimeSec,
            leafletColor     = vehicleProperties[v]['leafletColor'],
            cesiumColor      = vehicleProperties[v]['cesiumColor'], 
            routeType        = 'fastest',
            dataProvider     = 'ORS-online', 
            dataProviderArgs = {'APIkey': ORS_API_KEY})
        
        myAssignments = vrv.addStaticAssignment(
            initAssignments  = myAssignments,
            objectID         = v,
            modelFile        = vehicleProperties[v]['model'],
            loc              = list(myNodes[myNodes['id'] == arc[1]][['lat', 'lon']].values[0]),
            startTimeSec     = endTimeSec,
            endTimeSec       = endTimeSec + 30.0)
        
        endTimeSec += 30

        myAssignments = vrv.addStaticAssignment(
            initAssignments  = myAssignments,
            objectID         = 'package %s %d' % (v, arc[1]),
            modelFile        = '/veroviz/models/box_yellow.gltf',
            loc              = list(myNodes[myNodes['id'] == arc[1]][['lat', 'lon']].values[0]),
            startTimeSec     = endTimeSec,
            endTimeSec       = -1)        
myAssignments 

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,car,/veroviz/models/car_red.gltf,100,75,0.000000,42.887083,-78.873203,0,4.54919,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
1,1,car,/veroviz/models/car_red.gltf,100,75,4.549189,42.887044,-78.872978,0,6.8,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
2,1,car,/veroviz/models/car_red.gltf,100,75,6.800000,42.887031,-78.872865,0,7.46615,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
3,1,car,/veroviz/models/car_red.gltf,100,75,7.466153,42.887075,-78.872851,0,11.4302,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
4,1,car,/veroviz/models/car_red.gltf,100,75,11.430232,42.887336,-78.872763,0,27.617,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
5,1,car,/veroviz/models/car_red.gltf,100,75,27.616986,42.888402,-78.872405,0,28.3,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
6,1,car,/veroviz/models/car_red.gltf,100,75,28.300000,42.888447,-78.872390,0,28.945,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
7,1,car,/veroviz/models/car_red.gltf,100,75,28.945037,42.888434,-78.872315,0,37.5799,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
8,1,car,/veroviz/models/car_red.gltf,100,75,37.579881,42.888260,-78.871311,0,38.6451,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
9,1,car,/veroviz/models/car_red.gltf,100,75,38.645072,42.888239,-78.871187,0,39.4446,...,0,red,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8


In [16]:
# If you have saved your Cesium path as an environment variable, you may use `os.environ` to access it:
import os
CESIUM_DIR = os.environ['CESIUMDIR']
# Otherwise, you may specify the patch to Cesium here:
# CESIUM_DIR = '/provide/path/to/Cesium/' 

In [17]:
# Generate a 3D movie of the routes:
vrv.createCesium(assignments = myAssignments, 
                 nodes       = myNodes, 
                 cesiumDir   = CESIUM_DIR, 
                 problemDir  = 'addAssignment2D_example')

Message: File selector was written to /Users/murray/cesium/addAssignment2D_example/;addAssignment2D_example.vrv ...
Message: Configs were written to /Users/murray/cesium/addAssignment2D_example/config.js ...
Message: Nodes were written to /Users/murray/cesium/addAssignment2D_example/displayNodes.js ...
Message: Assignments (.js) were written to /Users/murray/cesium/addAssignment2D_example/displayPaths.js ...
Message: Assignments (.czml) were written to /Users/murray/cesium/addAssignment2D_example/routes.czml ...


### `addAssignment3D()`
This function appends to an existing `Assignments` dataframe, or creates a new `Assignments` dataframe if `initAssignments` is `None`.  The new rows in this dataframe describe all of the vehicle movements between given starting and ending locations, including timestamps indicating the departure and arrival times for each intermediate point. 
 
- See https://veroviz.org/docs/veroviz.createAssignments.html#veroviz.createAssignments.addAssignment3D for full documentation of this function.

In [1]:
# Import veroviz and check the latest version.
import veroviz as vrv

vrv.checkVersion()

'Your current installed version of veroviz is 0.2.3, the latest version available is 0.2.2. To update to the latest version, type `pip install --upgrade veroviz` at a command-line prompt.'

In [2]:
# Define 5 node locations, as [lat, lon] pairs:
locs = [[42.8871085, -78.8731949],
        [42.8888311, -78.8649649],
        [42.8802158, -78.8660787],
        [42.8845705, -78.8762794],
        [42.8908031, -78.8770140]]

In [3]:
# Generate a nodes dataframe from these locations:
myNodes = vrv.createNodesFromLocs(locs=locs)
myNodes

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,1,42.887108,-78.873195,0,None,None,glyphicon,info-sign,blue,1,pin,Cesium.Color.BLUE,1
1,2,42.888831,-78.864965,0,None,None,glyphicon,info-sign,blue,2,pin,Cesium.Color.BLUE,2
2,3,42.880216,-78.866079,0,None,None,glyphicon,info-sign,blue,3,pin,Cesium.Color.BLUE,3
3,4,42.884571,-78.876279,0,None,None,glyphicon,info-sign,blue,4,pin,Cesium.Color.BLUE,4
4,5,42.890803,-78.877014,0,None,None,glyphicon,info-sign,blue,5,pin,Cesium.Color.BLUE,5


In [4]:
# View these nodes on a map:
vrv.createLeaflet(nodes=myNodes)

Example 1 -- Assume a single drone delivers packages from node 1 to all other nodes.

In [ ]:
mySolution = {
    'drone': [[1,2,1], [1,3,1], [1,4,1], [1,5,1]]
}

In [ ]:
# Define some information about our drone, for later use:
vehicleProperties = {
    'drone': {'modelPackage': 'veroviz/models/drone_package.gltf',
              'modelEmpty': 'veroviz/models/drone.gltf',
              'leafletColor': 'red',
              'cesiumColor': 'Cesium.Color.RED'},
}

In [ ]:
# Build the assignments for the drone deliveries:
myAssignments = vrv.initDataframe('assignments')

endTimeSec = 0.0
for arc in mySolution['drone']:
    # Fly from i to j with a package:
    [myAssignments, endTimeSec] = vrv.addAssignment3D(
        initAssignments    = myAssignments,
        objectID           = 'drone',
        modelFile          = vehicleProperties['drone']['modelPackage'],
        startLoc           = list(myNodes[myNodes['id'] == arc[0]][['lat', 'lon']].values[0]),
        endLoc             = list(myNodes[myNodes['id'] == arc[1]][['lat', 'lon']].values[0]),
        startTimeSec       = endTimeSec,
        takeoffSpeedMPS    = vrv.convertSpeed(30, 'miles', 'hr', 'meters', 'sec'),
        cruiseSpeedMPS     = vrv.convertSpeed(80, 'miles', 'hr', 'meters', 'sec'),
        landSpeedMPS       = vrv.convertSpeed( 5, 'miles', 'hr', 'meters', 'sec'),
        cruiseAltMetersAGL = vrv.convertDistance(350, 'feet', 'meters'),
        routeType          = 'square',
        # climbRateMPS       = None,
        # descentRateMPS     = None,
        # earliestLandTime   = -1,
        # loiterPosition     = 'arrivalAtAlt',
        leafletColor       = vehicleProperties['drone']['leafletColor'],
        cesiumColor        = vehicleProperties['drone']['cesiumColor'])

    # Drop off a package
    myAssignments = vrv.addStaticAssignment(
            initAssignments  = myAssignments,
            objectID         = 'package %d' % (arc[1]),
            modelFile        = '/veroviz/models/box_yellow.gltf',
            loc              = list(myNodes[myNodes['id'] == arc[1]][['lat', 'lon']].values[0]),
            startTimeSec     = endTimeSec,
            endTimeSec       = -1)  
        
    # Fly from j to k empty:
    [myAssignments, endTimeSec] = vrv.addAssignment3D(
        initAssignments    = myAssignments,
        objectID           = 'drone',
        modelFile          = vehicleProperties['drone']['modelEmpty'],
        startLoc           = list(myNodes[myNodes['id'] == arc[1]][['lat', 'lon']].values[0]),
        endLoc             = list(myNodes[myNodes['id'] == arc[2]][['lat', 'lon']].values[0]),
        startTimeSec       = endTimeSec,
        takeoffSpeedMPS    = vrv.convertSpeed(30, 'miles', 'hr', 'meters', 'sec'),
        cruiseSpeedMPS     = vrv.convertSpeed(80, 'miles', 'hr', 'meters', 'sec'),
        landSpeedMPS       = vrv.convertSpeed( 5, 'miles', 'hr', 'meters', 'sec'),
        cruiseAltMetersAGL = vrv.convertDistance(350, 'feet', 'meters'),
        routeType          = 'square',
        # climbRateMPS       = None,
        # descentRateMPS     = None,
        # earliestLandTime   = -1,
        # loiterPosition     = 'arrivalAtAlt',
        leafletColor       = vehicleProperties['drone']['leafletColor'],
        cesiumColor        = vehicleProperties['drone']['cesiumColor'])

myAssignments

In [ ]:
# Show the nodes and assignments on a map:
vrv.createLeaflet(nodes=myNodes, arcs=myAssignments)

In [ ]:
# If you have saved your Cesium path as an environment variable, you may use `os.environ` to access it:
import os
CESIUM_DIR = os.environ['CESIUMDIR']
# Otherwise, you may specify the patch to Cesium here:
# CESIUM_DIR = '/provide/path/to/Cesium/' 

In [ ]:
# Create a 3D movie of the drone deliveries:
vrv.createCesium(assignments = myAssignments, 
                 nodes       = myNodes, 
                 cesiumDir   = CESIUM_DIR, 
                 problemDir  = 'addAssignment3D_example1')

Example 2 -- Coordinate deliveries with a drone launched from a truck

In [ ]:
# Hard-code a solution.
# The truck will visit nodes 1->3->5->1
# The drone will launch from the truck at node 1, deliver to node 2, and return to the truck at node 3.
# The drone will then launch from the truck at 1, deliver to 4 and return to 5.
# The drone cannot land at nodes 3 and 5 until the truck has arrived.
mySolution = {
    'drone': [[1,2,3], [3,4,5]],
    'truck': [[1,3], [3,5], [5,1]]
}

In [ ]:
# Define some information about our 2 vehicles, for use below:
vehicleProperties = {
    'drone': {'modelPackage': 'veroviz/models/drone_package.gltf',
              'modelEmpty': 'veroviz/models/drone.gltf',
              'leafletColor': 'red',
              'cesiumColor': 'Cesium.Color.RED'},
    'truck': {'model': 'veroviz/models/ub_truck.gltf',
              'leafletColor': 'blue',
              'cesiumColor': 'Cesium.Color.BLUE'}
}

In [ ]:
# This example assumes the use of ORS as the data provider. 
# If you have saved your API key as an environment variable, you may use `os.environ` to access it:
import os
ORS_API_KEY = os.environ['ORSKEY']
# Otherwise, you may specify your key here:
# ORS_API_KEY = 'YOUR_ORS_KEY_GOES_HERE'

In [ ]:
# Obtain the travel times for the truck:
[timeSecTruck, distMetersTruck] = vrv.getTimeDist2D(
    nodes            = myNodes,
    routeType        = 'fastest',
    dataProvider     = 'ORS-online',
    dataProviderArgs = {'APIkey': ORS_API_KEY})

In [ ]:
# Obtain the travel times for the drone:
[timeSecDrone, groundDist, TotalDist] = vrv.getTimeDist3D(
    nodes            = myNodes,
    takeoffSpeedMPS    = vrv.convertSpeed(30, 'miles', 'hr', 'meters', 'sec'),
    cruiseSpeedMPS     = vrv.convertSpeed(80, 'miles', 'hr', 'meters', 'sec'),
    landSpeedMPS       = vrv.convertSpeed( 5, 'miles', 'hr', 'meters', 'sec'),
    cruiseAltMetersAGL = vrv.convertDistance(350, 'feet', 'meters'),
    routeType          = 'square')

In [ ]:
# Find the coordination times for the truck and drone.
# These will be the earliest times that both the truck and drone can arrive at a node.
maxArrivalTime = {}

# 1) The truck travels from 1 to 3; the drone travels from 1 to 2 to 3:
truckArrivalTime = timeSecTruck[1,3]
droneArrivalTime = timeSecDrone[1,2] + timeSecDrone[2,3]
maxArrivalTime[3] = max(truckArrivalTime, droneArrivalTime)

# 2) The truck travels from 3 to 5; the drone travels from 3 to 4 to 5:
truckArrivalTime = maxArrivalTime[3] + timeSecTruck[3,5]
droneArrivalTime = maxArrivalTime[3] + timeSecDrone[3,4] + timeSecDrone[4,5]
maxArrivalTime[5] = max(truckArrivalTime, droneArrivalTime)

# 3) Let's also capture the time at which the truck will return to node 1:
maxArrivalTime[1] = maxArrivalTime[5] + timeSecTruck[5,1]

# maxArrival Time[1] is now the total time required to complete all of the deliveries
maxArrivalTime

In [ ]:
# Build assignments for the drone deliveries:
myAssignments = vrv.initDataframe('assignments')

endTimeSec = 0.0
for arc in mySolution['drone']:
    # Fly from i to j with a package:
    [myAssignments, endTimeSec] = vrv.addAssignment3D(
        initAssignments    = myAssignments,
        objectID           = 'drone',
        modelFile          = vehicleProperties['drone']['modelPackage'],
        startLoc           = list(myNodes[myNodes['id'] == arc[0]][['lat', 'lon']].values[0]),
        endLoc             = list(myNodes[myNodes['id'] == arc[1]][['lat', 'lon']].values[0]),
        startTimeSec       = endTimeSec,
        takeoffSpeedMPS    = vrv.convertSpeed(30, 'miles', 'hr', 'meters', 'sec'),
        cruiseSpeedMPS     = vrv.convertSpeed(80, 'miles', 'hr', 'meters', 'sec'),
        landSpeedMPS       = vrv.convertSpeed( 5, 'miles', 'hr', 'meters', 'sec'),
        cruiseAltMetersAGL = vrv.convertDistance(350, 'feet', 'meters'),
        routeType          = 'square',
        leafletColor       = vehicleProperties['drone']['leafletColor'],
        cesiumColor        = vehicleProperties['drone']['cesiumColor'])

    # Drop off a package
    myAssignments = vrv.addStaticAssignment(
            initAssignments  = myAssignments,
            objectID         = 'package %d' % (arc[1]),
            modelFile        = '/veroviz/models/box_yellow.gltf',
            loc              = list(myNodes[myNodes['id'] == arc[1]][['lat', 'lon']].values[0]),
            startTimeSec     = endTimeSec,
            endTimeSec       = -1)  
    
    # Fly from j to k empty:
    [myAssignments, endTimeSec] = vrv.addAssignment3D(
        initAssignments    = myAssignments,
        objectID           = 'drone',
        modelFile          = vehicleProperties['drone']['modelEmpty'],
        startLoc           = list(myNodes[myNodes['id'] == arc[1]][['lat', 'lon']].values[0]),
        endLoc             = list(myNodes[myNodes['id'] == arc[2]][['lat', 'lon']].values[0]),
        startTimeSec       = endTimeSec,
        takeoffSpeedMPS    = vrv.convertSpeed(30, 'miles', 'hr', 'meters', 'sec'),
        cruiseSpeedMPS     = vrv.convertSpeed(80, 'miles', 'hr', 'meters', 'sec'),
        landSpeedMPS       = vrv.convertSpeed( 5, 'miles', 'hr', 'meters', 'sec'),
        cruiseAltMetersAGL = vrv.convertDistance(350, 'feet', 'meters'),
        routeType          = 'square',
        earliestLandTime   = maxArrivalTime[arc[2]],
        loiterPosition     = 'arrivalAtAlt',
        leafletColor       = vehicleProperties['drone']['leafletColor'],
        cesiumColor        = vehicleProperties['drone']['cesiumColor'])

myAssignments

In [ ]:
# Build assignments for the truck route:
endTimeSec = 0.0
for arc in mySolution['truck']:
    [myAssignments, endTimeSec] = vrv.addAssignment2D(
            initAssignments  = myAssignments,
            objectID         = 'truck',
            modelFile        = vehicleProperties['truck']['model'],
            startLoc         = list(myNodes[myNodes['id'] == arc[0]][['lat', 'lon']].values[0]),
            endLoc           = list(myNodes[myNodes['id'] == arc[1]][['lat', 'lon']].values[0]),
            startTimeSec     = endTimeSec,
            leafletColor     = vehicleProperties['truck']['leafletColor'],
            cesiumColor      = vehicleProperties['truck']['cesiumColor'], 
            routeType        = 'fastest',
            dataProvider     = 'ORS-online', 
            dataProviderArgs = {'APIkey': ORS_API_KEY})
    
    # If necessary, wait for the drone to arrive:
    if (endTimeSec < maxArrivalTime[arc[1]]):
        myAssignments = vrv.addStaticAssignment(
            initAssignments  = myAssignments,
            objectID         = 'truck',
            modelFile        = vehicleProperties['truck']['model'],
            loc              = list(myNodes[myNodes['id'] == arc[1]][['lat', 'lon']].values[0]),
            startTimeSec     = endTimeSec,
            endTimeSec       = maxArrivalTime[arc[1]])

        endTimeSec = maxArrivalTime[arc[1]]  

In [ ]:
# Show the nodes and assignments on a map:
vrv.createLeaflet(nodes=myNodes, arcs=myAssignments)

In [ ]:
# Create a 3D movie of the vehicle routes:
vrv.createCesium(assignments = myAssignments, 
                 nodes       = myNodes, 
                 cesiumDir   = CESIUM_DIR, 
                 problemDir  = 'addAssignment3D_example2')

### `addStaticAssignment()`
This function adds an "object" to an assignments dataframe.  One use of this function is to add a package or other stationary 3D model to be displayed in Cesium.  This function is also useful for holding a vehicle stationary at a location (e.g., during service or loitering).  

- See https://veroviz.org/docs/veroviz.createAssignments.html#veroviz.createAssignments.addStaticAssignment for full documentation of this function.

In [3]:
# Import veroviz and check the latest version.
>>> import veroviz as vrv
>>> import os

>>> vrv.checkVersion()

'Your current installed version of veroviz is 0.2.3, the latest version available is 0.2.2. To update to the latest version, type `pip install --upgrade veroviz` at a command-line prompt.'

Create two nodes.
- See https://veroviz.org/docs/veroviz.generateNodes.html#veroviz.generateNodes.createNodesFromLocs for more info.

In [4]:
>>> myNodes = vrv.createNodesFromLocs(
...     locs = [[42.1538, -78.4253],
...             [42.6343, -78.1146]])
>>> myNodes

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,1,42.1538,-78.4253,0,None,None,glyphicon,info-sign,blue,1,pin,Cesium.Color.BLUE,1
1,2,42.6343,-78.1146,0,None,None,glyphicon,info-sign,blue,2,pin,Cesium.Color.BLUE,2


Move the truck from one node to the other.
- See https://veroviz.org/docs/veroviz.getShapepoints2D.html for more info.    

In [5]:
>>> myAssignments = vrv.getShapepoints2D(
...     odID           = 0,
...     objectID       = 'truck',
...     modelFile      = 'veroviz/models/ub_truck.gltf',
...     modelScale     = 80, 
...     modelMinPxSize = 20, 
...     startLoc       = list(myNodes.loc[0][['lat', 'lon']].values),
...     endLoc         = list(myNodes.loc[1][['lat', 'lon']].values),
...     routeType      = 'euclidean2D',
...     dataProvider   = None,
...     speedMPS       = vrv.convertSpeed(55, 'miles', 'hr', 'm', 's'))

>>> myAssignments

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,0,truck,veroviz/models/ub_truck.gltf,80,20,0.0,42.1538,-78.4253,0,2407.290832,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8


Make the truck wait at the destination.

In [6]:
>>> myAssignments = vrv.addStaticAssignment(
...     initAssignments = myAssignments, 
...     odID            = 0, 
...     objectID        = 'truck', 
...     modelFile       = 'veroviz/models/ub_truck.gltf', 
...     modelScale      = 80, 
...     modelMinPxSize  = 20, 
...     loc             = list(myAssignments[myAssignments['objectID']=='truck'][['endLat', 'endLon']].values[0]), 
...     startTimeSec    = myAssignments[myAssignments['objectID']=='truck']['endTimeSec'].values[0], 
...     endTimeSec      = myAssignments[myAssignments['objectID']=='truck']['endTimeSec'].values[0] + 30)

>>> myAssignments

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,0,truck,/veroviz/models/ub_truck.gltf,80,20,0.000000,42.1538,-78.4253,0,2407.290832,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
1,0,truck,veroviz/models/ub_truck.gltf,80,20,2407.290832,42.6343,-78.1146,0,2437.290832,...,0,None,None,None,None,None,None,None,None,None


Drop off a package.

In [7]:
>>> myAssignments = vrv.addStaticAssignment(
...     initAssignments = myAssignments, 
...     odID            = 0, 
...     objectID        = 'package', 
...     modelFile       = 'veroviz/models/box_blue.gltf', 
...     modelScale      = 50, 
...     modelMinPxSize  = 10, 
...     loc             = list(myAssignments[myAssignments['objectID']=='truck'][['endLat', 'endLon']].values[0]), 
...     startTimeSec    = myAssignments[myAssignments['objectID']=='truck']['endTimeSec'].values[0] + 30, 
...     endTimeSec      = -1)

>>> myAssignments

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,0,truck,/veroviz/models/ub_truck.gltf,80,20,0.000000,42.1538,-78.4253,0,2407.29,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
1,0,truck,/veroviz/models/ub_truck.gltf,80,20,2407.290832,42.6343,-78.1146,0,2437.29,...,0,None,None,None,None,None,None,None,None,None
2,0,package,veroviz/models/box_blue.gltf,50,10,2437.290832,42.6343,-78.1146,0,-1,...,0,None,None,None,None,None,None,None,None,None


View in Leaflet.  
- Note that we can't see the truck waiting. Also, the package doesn't show up in Leaflet.
- See https://veroviz.org/docs/veroviz.createLeaflet.html for more info.

In [8]:
vrv.createLeaflet(nodes=myNodes, arcs=myAssignments)

Create Cesium output, so we can observe our truck and package.
- See https://veroviz.org/docs/veroviz.createCesium.html for more info.

In [9]:
>>> vrv.createCesium(
...     assignments = myAssignments, 
...     nodes       = myNodes, 
...     startTime   = '08:00:00', 
...     cesiumDir   = os.environ['CESIUMDIR'],
...     problemDir  = 'static_object_demo')

Message: File selector was written to /Users/murray/cesium/static_object_demo/;static_object_demo.vrv ...
Message: Configs were written to /Users/murray/cesium/static_object_demo/config.js ...
Message: Nodes were written to /Users/murray/cesium/static_object_demo/displayNodes.js ...
Message: Assignments (.js) were written to /Users/murray/cesium/static_object_demo/displayPaths.js ...
Message: Assignments (.czml) were written to /Users/murray/cesium/static_object_demo/routes.czml ...


### `createAssignmentsFromArcs2D()`
This function generates an "assignments" dataframe containing all of the "shapepoints" between successive arcs, including timestamps indicating the departure and arrival times for each shapepoint. Shapepoints are pairs of GPS coordinates that are connected by straight lines.  For a particular origin and destination, numerous individual shapepoints can be combined to define a travel route along a road network. 

- This function is for vehicles traveling on a ground plane (2-dimensional).  For vehicles requiring an altitude component (e.g., drones), a 3D version of this function is provided by `createAssignmentsFromArcs3D()`.
	
- This function creates an assignments dataframe from an arcs dataframe.  Similar functions are available to create an assignments dataframe from a sequence of nodes (`createShapepointsFromNodeSeq2D()`) or from a sequence of locations (`createShapepointsFromLocSeq2D()`).

- See https://veroviz.org/docs/veroviz.createAssignments.html#veroviz.createAssignments.createAssignmentsFromArcs2D for full documentation of this function.

In [ ]:
# First import veroviz and check if it is the latest version
import veroviz as vrv
vrv.checkVersion()

In [ ]:
# Generate arcs from a given ordered list of coordinates:
arcs = vrv.createArcsFromLocSeq(
    locSeq=[[42.3538, -78.4253, 30], 
            [42.3465, -78.4234, 30], 
            [42.3343, -78.4146, 40]])
arcs

In [ ]:
# Display the arcs on a Leaflet map:
vrv.createLeaflet(arcs=arcs)

In [ ]:
# The following examples assume the use of ORS as the data provider. 
# If you have saved your API key as an environment variable, 
# you may use `os.environ` to access it:
import os
ORS_API_KEY = os.environ['ORSKEY']
# Otherwise, you may specify your key here:
# ORS_API_KEY = 'YOUR_ORS_KEY_GOES_HERE'

In [ ]:
# Generate an assignments dataframe from the arcs dataframe:
myAssignments = vrv.createAssignmentsFromArcs2D(
    arcs             = arcs, 
    modelFile        = 'veroviz/models/car_blue.gltf', 
    routeType        = 'fastest', 
    dataProvider     = 'ors-online',
    dataProviderArgs = {'APIkey': ORS_API_KEY},
    leafletColor     = 'blue')
myAssignments

In [ ]:
# Display the assignments on a map:
vrv.createLeaflet(arcs=myAssignments)

### `createAssignmentsFromLocSeq2D()`
This function generates an "assignments" dataframe containing all of the "shapepoints" between successive locations, including timestamps indicating the departure and arrival times for each shapepoint. Shapepoints are pairs of GPS coordinates that are connected by straight lines.  For a particular origin and destination, numerous individual shapepoints can be combined to define a travel route along a road network.  

- This function is for vehicles traveling on a ground plane (2-dimensional).  For vehicles requiring an altitude component (e.g., drones), a 3D version of this function is provided by `createAssignmentsFromLocSeq3D()`.
	
- This function creates an assignments dataframe from a sequence of [lat, lon] locations.  Similar functions are available to create an assignments dataframe from an arcs dataframe (`createShapepointsFromArcs2D()`) or from a sequence of nodes (`createShapepointsFromNodeSeq2D()`).

- See https://veroviz.org/docs/veroviz.createAssignments.html#veroviz.createAssignments.createAssignmentsFromLocSeq2D for full documentation of this function.

In [10]:
# Import veroviz and check the latest version.
>>> import veroviz as vrv

>>> vrv.checkVersion()

'Your current installed version of veroviz is 0.2.3, the latest version available is 0.2.2. To update to the latest version, type `pip install --upgrade veroviz` at a command-line prompt.'

In [11]:
# The following examples assume the use of ORS as the data provider. If you have saved your API key as an environment variable, you may use `os.environ` to access it:
>>> import os
>>> ORS_API_KEY = os.environ['ORSKEY']
>>> # Otherwise, you may specify your key here:
>>> # ORS_API_KEY = 'YOUR_ORS_KEY_GOES_HERE'

In [12]:
# Specify a sequence of [lat, lon] or [lat, lon, alt] locations. 
>>> locs = [
...     [42.1538, -78.4253, 30], 
...     [42.3465, -78.6234, 30], 
...     [42.6343, -78.1146, 40],
...     [42.1538, -78.4253, 30]] 

Example 1 - A simple example using Euclidean travel, and no service times.  The assignments dataframe will have 3 rows.

In [13]:
>>> assignmentsDF = vrv.createAssignmentsFromLocSeq2D(
...     locSeq         = locs, 
...     serviceTimeSec = 0.0, 
...     objectID       = 'Blue Car', 
...     modelFile      = 'veroviz/models/car_blue.gltf', 
...     routeType      = 'euclidean2D',
...     speedMPS       = 10)
>>> assignmentsDF

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,Blue Car,veroviz/models/car_blue.gltf,100,75,0.000000,42.1538,-78.4253,30,2693.379882,...,30,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
1,2,Blue Car,veroviz/models/car_blue.gltf,100,75,2693.379882,42.3465,-78.6234,30,7958.098133,...,40,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
2,3,Blue Car,veroviz/models/car_blue.gltf,100,75,7958.098133,42.6343,-78.1146,40,13876.937537,...,30,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8


Example 2 - The vehicle will follow the road network, resulting in an assignments dataframe with significantly more than 3 rows.  The vehicle will remain stationary at the destination locations.  The travel time for each origin/destination pair will be calculated separately.

In [14]:
>>> # Make our car wait for 65 seconds before starting:
>>> assignmentsDF = vrv.addStaticAssignment(
...     odID         = 0, 
...     objectID     = 'Blue Car', 
...     modelFile    = 'veroviz/models/car_blue.gltf', 
...     loc          = locs[0], 
...     startTimeSec = 0.0, 
...     endTimeSec   = 65.0)

In [15]:
>>> # This example includes all of the available input arguments:
>>> newAssignmentsDF = vrv.createAssignmentsFromLocSeq2D(
...     initAssignments  = assignmentsDF, 
...     locSeq           = locs, 
...     serviceTimeSec   = 20.0, 
...     odID             = 1, 
...     objectID         = 'Blue Car', 
...     modelFile        = 'veroviz/models/car_blue.gltf', 
...     modelScale       = 100,
...     modelMinPxSize   = 75,
...     startTimeSec     = 65.0, 
...     expDurationArgs  = {'getTravelTimes': True}, 
...     routeType        = 'fastest',
...     speedMPS         = None,   
...     leafletColor     = 'blue', 
...     leafletWeight    = 3, 
...     leafletStyle     = 'dashed', 
...     leafletOpacity   = 0.8, 
...     useArrows        = True, 
...     cesiumColor      = 'Cesium.Color.BLUE', 
...     cesiumWeight     = 3, 
...     cesiumStyle      = 'solid', 
...     cesiumOpacity    = 0.8, 
...     dataProvider     = 'ORS-online',
...     dataProviderArgs = {'APIkey' : ORS_API_KEY})
>>> newAssignmentsDF


Message: The origin point (lat: 42.1538, lon: -78.4253) is 492.9 meters away from the road. You might find a gap between the origin point and the route.
Message: The destination point (lat: 42.3465, lon: -78.6234) is 254.8 meters away from the road. You might find a gap between destination point and the route.
Message: The origin point (lat: 42.3465, lon: -78.6234) is 254.8 meters away from the road. You might find a gap between the origin point and the route.
Message: The destination point (lat: 42.6343, lon: -78.1146) is 343.9 meters away from the road. You might find a gap between destination point and the route.
Message: The origin point (lat: 42.6343, lon: -78.1146) is 343.9 meters away from the road. You might find a gap between the origin point and the route.
Message: The destination point (lat: 42.1538, lon: -78.4253) is 492.9 meters away from the road. You might find a gap between destination point and the route.


,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,0,Blue Car,/veroviz/models/car_blue.gltf,100,75,0.000000,42.153800,-78.425300,30,65.000000,...,30,None,None,None,None,None,None,None,None,None
1,1,Blue Car,veroviz/models/car_blue.gltf,100,75,65.000000,42.157210,-78.421483,0,79.345202,...,0,blue,3,dashed,0.8,True,Cesium.Color.BLUE,3,solid,0.8
2,1,Blue Car,veroviz/models/car_blue.gltf,100,75,79.345202,42.158075,-78.422889,0,81.887353,...,0,blue,3,dashed,0.8,True,Cesium.Color.BLUE,3,solid,0.8
3,1,Blue Car,veroviz/models/car_blue.gltf,100,75,81.887353,42.158237,-78.423128,0,90.133750,...,0,blue,3,dashed,0.8,True,Cesium.Color.BLUE,3,solid,0.8
4,1,Blue Car,veroviz/models/car_blue.gltf,100,75,90.133750,42.158812,-78.423837,0,98.796565,...,0,blue,3,dashed,0.8,True,Cesium.Color.BLUE,3,solid,0.8
5,1,Blue Car,veroviz/models/car_blue.gltf,100,75,98.796565,42.159465,-78.424503,0,114.381873,...,0,blue,3,dashed,0.8,True,Cesium.Color.BLUE,3,solid,0.8
6,1,Blue Car,veroviz/models/car_blue.gltf,100,75,114.381873,42.160656,-78.425672,0,119.179024,...,0,blue,3,dashed,0.8,True,Cesium.Color.BLUE,3,solid,0.8
7,1,Blue Car,veroviz/models/car_blue.gltf,100,75,119.179024,42.161050,-78.425975,0,136.568916,...,0,blue,3,dashed,0.8,True,Cesium.Color.BLUE,3,solid,0.8
8,1,Blue Car,veroviz/models/car_blue.gltf,100,75,136.568916,42.162560,-78.426854,0,147.209515,...,0,blue,3,dashed,0.8,True,Cesium.Color.BLUE,3,solid,0.8
9,1,Blue Car,veroviz/models/car_blue.gltf,100,75,147.209515,42.163508,-78.427310,0,156.909447,...,0,blue,3,dashed,0.8,True,Cesium.Color.BLUE,3,solid,0.8


In [17]:
# Generate a `Nodes` dataframe from our first three locations 
# (the fourth location is a duplicate of the first).  
exampleNodes = vrv.createNodesFromLocs(locs=locs[0:3])

In [18]:
# View the assignments in Leaflet:  
>>> vrv.createLeaflet(arcs=newAssignmentsDF, nodes=exampleNodes)

In [19]:
# View the assignments in Cesium:
>>> vrv.createCesium(
...     assignments = newAssignmentsDF, 
...     nodes       = exampleNodes, 
...     startTime   = '08:00:00', 
...     cesiumDir   = os.environ['CESIUMDIR'],
...     problemDir  = 'createAssignments_example')

Message: File selector was written to /Users/murray/cesium/createAssignments_example/;createAssignments_example.vrv ...
Message: Configs were written to /Users/murray/cesium/createAssignments_example/config.js ...
Message: Nodes were written to /Users/murray/cesium/createAssignments_example/displayNodes.js ...
Message: Assignments (.js) were written to /Users/murray/cesium/createAssignments_example/displayPaths.js ...
Message: Assignments (.czml) were written to /Users/murray/cesium/createAssignments_example/routes.czml ...


### `createAssignmentsFromNodeSeq2D()`
This function generates an "assignments" dataframe containing all of the "shapepoints" between successive node locations, including timestamps indicating the departure and arrival times for each shapepoint. Shapepoints are pairs of GPS coordinates that are connected by straight lines.  For a particular origin and destination, numerous individual shapepoints can be combined to define a travel route along a road network. 

- This function is for vehicles traveling on a ground plane (2-dimensional).  For vehicles requiring an altitude component (e.g., drones), a 3D version of this function is provided by `createAssignmentsFromLocSeq3D()`.
	
- This function creates an assignments dataframe from a sequence of nodes.  Similar functions are available to create an assignments dataframe from an arcs dataframe (`createShapepointsFromArcs2D()`) or from a sequence of locations (`createShapepointsFromLocSeq2D()`).

- See https://veroviz.org/docs/veroviz.createAssignments.html#veroviz.createAssignments.createAssignmentsFromNodeSeq2D for full documentation of this function.

In [20]:
# Import veroviz and check the latest version.
>>> import veroviz as vrv

>>> vrv.checkVersion()

'Your current installed version of veroviz is 0.2.3, the latest version available is 0.2.2. To update to the latest version, type `pip install --upgrade veroviz` at a command-line prompt.'

In [21]:
# The following examples assume the use of ORS as the data provider. If you have saved your API key as an environment variable, you may use `os.environ` to access it:
>>> import os
>>> ORS_API_KEY = os.environ['ORSKEY']
>>> # Otherwise, you may specify your key here:
>>> # ORS_API_KEY = 'YOUR_ORS_KEY_GOES_HERE'

In [22]:
# Generate a `Nodes` dataframe from a list of coordinates.  
>>> locs = [
...     [42.1538, -78.4253], 
...     [42.3465, -78.6234], 
...     [42.6343, -78.1146]]
>>> exampleNodes = vrv.createNodesFromLocs(locs=locs)

Example 1 - A simple example using Euclidean travel, and no service times.  The assignments dataframe will have 3 rows.

In [23]:
>>> assignmentsDF = vrv.createAssignmentsFromNodeSeq2D(
...     nodeSeq        = [1, 3, 2, 1], 
...     nodes          = exampleNodes, 
...     serviceTimeSec = 0.0, 
...     objectID       = 'Blue Car', 
...     modelFile      = 'veroviz/models/car_blue.gltf', 
...     routeType      = 'euclidean2D',
...     speedMPS       = 10)
>>> assignmentsDF

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,Blue Car,veroviz/models/car_blue.gltf,100,75,0.000000,42.1538,-78.4253,0,5918.839403,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
1,2,Blue Car,veroviz/models/car_blue.gltf,100,75,5918.839403,42.6343,-78.1146,0,11183.557655,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
2,3,Blue Car,veroviz/models/car_blue.gltf,100,75,11183.557655,42.3465,-78.6234,0,13876.937537,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8


Example 2 - The vehicle will follow the road network, resulting in an assignments dataframe with significantly more than 3 rows.  The vehicle will remain stationary at nodes 3, 2, and 1 (the destination nodes).  The travel time for each origin/destination pair will be calculated separately.

In [24]:
>>> assignmentsDF = vrv.createAssignmentsFromNodeSeq2D(
...     initAssignments  = None, 
...     nodeSeq          = [1, 3, 2, 1], 
...     nodes            = exampleNodes, 
...     serviceTimeSec   = 20.0, 
...     odID             = 1, 
...     objectID         = 'Blue Car', 
...     modelFile        = 'veroviz/models/car_blue.gltf', 
...     expDurationArgs  = {'getTravelTimes': True}, 
...     routeType        = 'fastest',
...     dataProvider     = 'ORS-online',
...     dataProviderArgs = {'APIkey' : ORS_API_KEY})
>>> assignmentsDF


Message: The origin point (lat: 42.1538, lon: -78.4253) is 492.9 meters away from the road. You might find a gap between the origin point and the route.
Message: The destination point (lat: 42.6343, lon: -78.1146) is 343.9 meters away from the road. You might find a gap between destination point and the route.
Message: The origin point (lat: 42.6343, lon: -78.1146) is 343.9 meters away from the road. You might find a gap between the origin point and the route.
Message: The destination point (lat: 42.3465, lon: -78.6234) is 254.8 meters away from the road. You might find a gap between destination point and the route.
Message: The origin point (lat: 42.3465, lon: -78.6234) is 254.8 meters away from the road. You might find a gap between the origin point and the route.
Message: The destination point (lat: 42.1538, lon: -78.4253) is 492.9 meters away from the road. You might find a gap between destination point and the route.


,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,Blue Car,veroviz/models/car_blue.gltf,100,75,0.000000,42.157210,-78.421483,0,2.489840,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
1,1,Blue Car,veroviz/models/car_blue.gltf,100,75,2.489840,42.156979,-78.421107,0,9.602194,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
2,1,Blue Car,veroviz/models/car_blue.gltf,100,75,9.602194,42.156253,-78.420113,0,11.153721,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
3,1,Blue Car,veroviz/models/car_blue.gltf,100,75,11.153721,42.156079,-78.419919,0,13.319079,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
4,1,Blue Car,veroviz/models/car_blue.gltf,100,75,13.319079,42.155856,-78.419619,0,15.245843,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
5,1,Blue Car,veroviz/models/car_blue.gltf,100,75,15.245843,42.155672,-78.419334,0,18.572607,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
6,1,Blue Car,veroviz/models/car_blue.gltf,100,75,18.572607,42.155378,-78.418816,0,24.052151,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
7,1,Blue Car,veroviz/models/car_blue.gltf,100,75,24.052151,42.154778,-78.418108,0,41.846543,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
8,1,Blue Car,veroviz/models/car_blue.gltf,100,75,41.846543,42.152856,-78.415769,0,50.910191,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
9,1,Blue Car,veroviz/models/car_blue.gltf,100,75,50.910191,42.151901,-78.414543,0,54.971046,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8


Example 3 - The vehicle will first wait at the starting location (node 1) for 65 seconds.  It will then visit nodes 3 and 2 before returning to node 1.  At each of those nodes, the vehicle will remain stationary for a 20-second service time.  The travel time between each pair of nodes will be determined by the times in the `timeSec` matrix.

In [25]:
# Generate a matrix of travel times:
>>> [timeSec, distMeters] = vrv.getTimeDist2D(
...     nodes            = exampleNodes,
...     matrixType       = 'all2all',
...     outputDistUnits  = 'meters',
...     outputTimeUnits  = 'seconds',
...     routeType        = 'fastest',
...     speedMPS         = None,   
...     dataProvider     = 'ORS-online',
...     dataProviderArgs = {'APIkey' : ORS_API_KEY})

In [26]:
# Make our car wait for 65 seconds before starting:
>>> assignmentsDF = vrv.addStaticAssignment(
...     odID         = 0, 
...     objectID     = 'Blue Car', 
...     modelFile    = 'veroviz/models/car_blue.gltf', 
...     loc          = [exampleNodes.loc[exampleNodes['id'] == 1]['lat'].values[0],
...                     exampleNodes.loc[exampleNodes['id'] == 1]['lon'].values[0]], 
...     startTimeSec = 0.0, 
...     endTimeSec   = 65.0)

In [27]:
# This example includes all of the available input arguments:
>>> newAssignmentsDF = vrv.createAssignmentsFromNodeSeq2D(
...     initAssignments  = assignmentsDF, 
...     nodeSeq          = [1, 3, 2, 1], 
...     nodes            = exampleNodes, 
...     serviceTimeSec   = 20.0, 
...     odID             = 1, 
...     objectID         = 'Blue Car', 
...     modelFile        = 'veroviz/models/car_blue.gltf', 
...     modelScale       = 100,
...     modelMinPxSize   = 75,
...     startTimeSec     = 65.0, 
...     expDurationArgs  = {'timeSecDict': timeSec}, 
...     routeType        = 'fastest',
...     speedMPS         = None,   
...     leafletColor     = 'blue', 
...     leafletWeight    = 3, 
...     leafletStyle     = 'dashed', 
...     leafletOpacity   = 0.8, 
...     useArrows        = True, 
...     cesiumColor      = 'Cesium.Color.BLUE', 
...     cesiumWeight     = 3, 
...     cesiumStyle      = 'solid', 
...     cesiumOpacity    = 0.8, 
...     dataProvider     = 'ORS-online',
...     dataProviderArgs = {'APIkey' : ORS_API_KEY})
>>> newAssignmentsDF


Message: The origin point (lat: 42.1538, lon: -78.4253) is 492.9 meters away from the road. You might find a gap between the origin point and the route.
Message: The destination point (lat: 42.6343, lon: -78.1146) is 343.9 meters away from the road. You might find a gap between destination point and the route.
Message: The origin point (lat: 42.6343, lon: -78.1146) is 343.9 meters away from the road. You might find a gap between the origin point and the route.
Message: The destination point (lat: 42.3465, lon: -78.6234) is 254.8 meters away from the road. You might find a gap between destination point and the route.
Message: The origin point (lat: 42.3465, lon: -78.6234) is 254.8 meters away from the road. You might find a gap between the origin point and the route.
Message: The destination point (lat: 42.1538, lon: -78.4253) is 492.9 meters away from the road. You might find a gap between destination point and the route.


,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,0,Blue Car,/veroviz/models/car_blue.gltf,100,75,0.000000,42.153800,-78.425300,0,65.000000,...,0,None,None,None,None,None,None,None,None,None
1,1,Blue Car,veroviz/models/car_blue.gltf,100,75,65.000000,42.157210,-78.421483,0,67.489840,...,0,blue,3,dashed,0.8,True,Cesium.Color.BLUE,3,solid,0.8
2,1,Blue Car,veroviz/models/car_blue.gltf,100,75,67.489840,42.156979,-78.421107,0,74.602194,...,0,blue,3,dashed,0.8,True,Cesium.Color.BLUE,3,solid,0.8
3,1,Blue Car,veroviz/models/car_blue.gltf,100,75,74.602194,42.156253,-78.420113,0,76.153721,...,0,blue,3,dashed,0.8,True,Cesium.Color.BLUE,3,solid,0.8
4,1,Blue Car,veroviz/models/car_blue.gltf,100,75,76.153721,42.156079,-78.419919,0,78.319079,...,0,blue,3,dashed,0.8,True,Cesium.Color.BLUE,3,solid,0.8
5,1,Blue Car,veroviz/models/car_blue.gltf,100,75,78.319079,42.155856,-78.419619,0,80.245843,...,0,blue,3,dashed,0.8,True,Cesium.Color.BLUE,3,solid,0.8
6,1,Blue Car,veroviz/models/car_blue.gltf,100,75,80.245843,42.155672,-78.419334,0,83.572607,...,0,blue,3,dashed,0.8,True,Cesium.Color.BLUE,3,solid,0.8
7,1,Blue Car,veroviz/models/car_blue.gltf,100,75,83.572607,42.155378,-78.418816,0,89.052151,...,0,blue,3,dashed,0.8,True,Cesium.Color.BLUE,3,solid,0.8
8,1,Blue Car,veroviz/models/car_blue.gltf,100,75,89.052151,42.154778,-78.418108,0,106.846543,...,0,blue,3,dashed,0.8,True,Cesium.Color.BLUE,3,solid,0.8
9,1,Blue Car,veroviz/models/car_blue.gltf,100,75,106.846543,42.152856,-78.415769,0,115.910191,...,0,blue,3,dashed,0.8,True,Cesium.Color.BLUE,3,solid,0.8


In [28]:
# View the assignments in Leaflet:  
>>> vrv.createLeaflet(arcs=newAssignmentsDF, nodes=exampleNodes)

In [29]:
# View the assignments in Cesium:
>>> vrv.createCesium(
...     assignments = newAssignmentsDF, 
...     nodes       = exampleNodes, 
...     startTime   = '08:00:00', 
...     cesiumDir   = os.environ['CESIUMDIR'],
...     problemDir  = 'createAssignments_example')

Message: File selector was written to /Users/murray/cesium/createAssignments_example/;createAssignments_example.vrv ...
Message: Configs were written to /Users/murray/cesium/createAssignments_example/config.js ...
Message: Nodes were written to /Users/murray/cesium/createAssignments_example/displayNodes.js ...
Message: Assignments (.js) were written to /Users/murray/cesium/createAssignments_example/displayPaths.js ...
Message: Assignments (.czml) were written to /Users/murray/cesium/createAssignments_example/routes.czml ...


--- 

## createCesium Module

This module includes the following function:
- `createCesium()`

### `createCesium()`

This function generates several files required to view a solution in Cesium. The function requires assignments and/or nodes dataframes as input.

- See https://veroviz.org/docs/veroviz.createCesium.html#module-veroviz.createCesium for complete function documentation.
- See https://veroviz.org/downloads/cesium_demo.ipynb.zip for a downloadable example.

In [1]:
# Import veroviz and check the latest version.
>>> import veroviz as vrv
>>> import os

>>> vrv.checkVersion()

'Your current installed version of veroviz is 0.2.0. You are up-to-date with the latest available version.'

In [2]:
# Create two nodes.
>>> myNodes = vrv.createNodesFromLocs(
...     locs = [[42.1538, -78.4253],
...             [42.6343, -78.1146]])
>>> myNodes

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,1,42.1538,-78.4253,0,None,None,glyphicon,info-sign,blue,1,pin,Cesium.Color.BLUE,1
1,2,42.6343,-78.1146,0,None,None,glyphicon,info-sign,blue,2,pin,Cesium.Color.BLUE,2


In [3]:
# Move the truck from one node to the other.
>>> myAssignments = vrv.getShapepoints2D(
...     odID           = 0,
...     objectID       = 'truck',
...     modelFile      = 'veroviz/models/ub_truck.gltf',
...     modelScale     = 80, 
...     modelMinPxSize = 20, 
...     startLoc       = list(myNodes.loc[0][['lat', 'lon']].values),
...     endLoc         = list(myNodes.loc[1][['lat', 'lon']].values),
...     routeType      = 'euclidean2D',
...     dataProvider   = None,
...     speedMPS       = vrv.convertSpeed(55, 'miles', 'hr', 'm', 's'))

In [4]:
# Create Cesium output.
>>> vrv.createCesium(
...     assignments = myAssignments, 
...     nodes       = myNodes, 
...     startTime   = '08:00:00', 
...     cesiumDir   = os.environ['CESIUMDIR'],
...     problemDir  = 'createCesium_example')

Message: File selector was written to /Users/murray/cesium/createCesium_example/;createCesium_example.vrv ...
Message: Configs were written to /Users/murray/cesium/createCesium_example/config.js ...
Message: Nodes were written to /Users/murray/cesium/createCesium_example/displayNodes.js ...
Message: Assignments (.js) were written to /Users/murray/cesium/createCesium_example/displayPaths.js ...
Message: Assignments (.czml) were written to /Users/murray/cesium/createCesium_example/routes.czml ...


--- 

## createLeaflet Module

This module includes the following functions:
- `addLeafletCircle()`
- `addLeafletMarker()`
- `addLeafletPolygon()`
- `addLeafletPolyline()`
- `addLeafletText()`
- `createLeaflet()`

### `addLeafletCircle()`
Add a circle, with a radius specified in [meters], to a Leaflet map.

- See https://veroviz.org/docs/veroviz.createLeaflet.html#veroviz.createLeaflet.addLeafletCircle for complete function documentation.

In [30]:
>>> # Draw a circle of radius 10 meters, centered on the Univ. at Buffalo campus.
>>> # Save this as "a_circle.html".
>>> import veroviz as vrv
>>> myMap = vrv.addLeafletCircle(
...     zoomStart=18,
...     center=[43.00154, -78.7871],
...     radius=100,
...     mapFilename="a_circle.html")
>>> myMap

Message: Map page written to a_circle.html.


In [31]:
>>> # Draw a circle of radius 2000 meters, centered on the Univ. at Buffalo campus.
>>> # This example includes all of the available function arguments.
>>> import veroviz as vrv
>>> myMap = vrv.addLeafletCircle(
...     mapObject = None,
...     mapFilename = None,
...     mapBackground = 'OpenStreetMap',
...     mapBoundary = None,
...     zoomStart = 13,
...     center = [43.00154, -78.7871],
...     radius = 2000,
...     lineWeight = 6,
...     lineColor = '#ff66ff',
...     lineOpacity = 0.7,
...     lineStyle = 'dotted',
...     fillColor = 'green',
...     fillOpacity = 0.4)
>>> myMap

### `addLeafletMarker()`
Add a circle-shaped marker, with a radius specified in [pixels], to a Leaflet map.

- See https://veroviz.org/docs/veroviz.createLeaflet.html#veroviz.createLeaflet.addLeafletMarker for complete function documentation.

In [32]:
>>> # Draw a circle of radius 10 pixels, centered on the Univ. at Buffalo campus.
>>> # Save this as "a_marker.html".
>>> import veroviz as vrv
>>> myMap = vrv.addLeafletMarker(
...     center=[43.00154, -78.7871],
...     radius=10,
...     mapFilename="a_marker.html")
>>> myMap

Message: Map page written to a_marker.html.


In [33]:
>>> # Draw a circle of radius 30 pixels, centered on the Univ. at Buffalo campus.
>>> # This example includes all of the available function arguments.
>>> import veroviz as vrv
>>> myMap = vrv.addLeafletMarker(
...     mapObject = None, 
...     mapFilename = None, 
...     mapBackground = 'CartoDB positron', 
...     mapBoundary = None, 
...     zoomStart = 11, 
...     center = [43.00154, -78.7871],
...     radius = 30, 
...     lineWeight = 3, 
...     lineColor = 'orange', 
...     lineOpacity = 0.6, 
...     lineStyle = 'dashed',
...     fillColor = 'blue', 
...     fillOpacity = 0.3)
>>> myMap

### `addLeafletPolygon()`
Add a polygon, as defined by an ordered collection of lat/lon coordinates, to a Leaflet map.

- See https://veroviz.org/docs/veroviz.createLeaflet.html#veroviz.createLeaflet.addLeafletPolygon for complete function documentation.

In [34]:
>>> # Draw a filled polygon around the Univ. at Buffalo campus.
>>> # Save this as "a_polygon.html".
>>> import veroviz as vrv
>>> campusPoints = [[43.0121, -78.7858],
...                 [43.0024, -78.7977],
...                 [42.9967, -78.7921],
...                 [42.9988, -78.7790]]
>>> myMap = vrv.addLeafletPolygon(
...     points=campusPoints,
...     mapFilename="a_polygon.html")
>>> myMap

Message: Map page written to a_polygon.html.


In [35]:
>>> # Draw a filled polygon around the Univ. at Buffalo campus.
>>> # This example includes all of the available function arguments.
>>> import veroviz as vrv
>>> campusPoints = [[43.0121, -78.7858],
...                 [43.0024, -78.7977],
...                 [42.9967, -78.7921],
...                 [42.9988, -78.7790]]
>>> myMap = vrv.addLeafletPolygon(
...     mapObject = None, 
...     mapFilename = None, 
...     mapBackground = 'OpenStreetMap', 
...     mapBoundary = vrv.getMapBoundary(locs=campusPoints), 
...     zoomStart = 15, 
...     points = campusPoints, 
...     lineWeight = 7, 
...     lineColor = '#ff00ff', 
...     lineOpacity = 0.9, 
...     lineStyle = 'solid', 
...     fillColor = '#ff66ff', 
...     fillOpacity = 0.3)    
>>> myMap

### `addLeafletPolyline()`
Add a polyline, as described by an ordered collection of lat/lon coordinates, to a Leaflet map.  The polyline is an "open" shape, in the sense that there's nothing connecting the first and last locations.  By contrast, the "polygon" shape will automatically connect the first and last locations.

- See https://veroviz.org/docs/veroviz.createLeaflet.html#veroviz.createLeaflet.addLeafletPolyline for complete function documentation.

In [36]:
>>> # Draw a polyline around the northern portion of the Univ. at Buffalo campus.
>>> # Save this as "a_polyline.html".
>>> import veroviz as vrv
>>> campusPoints = [[43.0024, -78.7977],
...                 [43.0121, -78.7858],
...                 [42.9988, -78.7790]]
>>> myMap = vrv.addLeafletPolyline(
...     points=campusPoints,
...     mapFilename="a_polyline.html")
>>> myMap

Message: Map page written to a_polyline.html.


In [37]:
>>> # Draw a polyline around the northern portion of the Univ. at Buffalo campus.
>>> # This example includes all of the available function arguments.
>>> import veroviz as vrv
>>> campusPoints = [[43.0024, -78.7977],
...                 [43.0121, -78.7858],
...                 [42.9988, -78.7790]]
>>> myMap = vrv.addLeafletPolyline(
...     mapObject = None, 
...     mapFilename = None, 
...     mapBackground = 'CartoDB positron', 
...     mapBoundary = vrv.getMapBoundary(locs=campusPoints),
...     zoomStart = 14, 
...     points = campusPoints,
...     lineWeight = 13, 
...     lineColor = '#0055ff', 
...     lineOpacity = 0.8, 
...     lineStyle = 'solid')
>>> myMap

### `addLeafletText()`
Add a text label to a Leaflet map. 

- See https://veroviz.org/docs/veroviz.createLeaflet.html#veroviz.createLeaflet.addLeafletText for complete function documentation.

In [38]:
>>> # Draw a text label at the location of Bell Hall on the
>>> # Univ. at Buffalo campus.
>>> # Save this as "a_text_label.html".
>>> import veroviz as vrv
>>> myMap = vrv.addLeafletText(
...     anchorPoint=[43.00154, -78.7871],
...     text="Bell Hall",
...     mapFilename="a_text_label.html")
>>> myMap

Message: Map page written to a_text_label.html.


In [39]:
>>> # Draw a text label at the location of Bell Hall on the
>>> # Univ. at Buffalo campus.
>>> # This example includes all of the available function arguments.
>>> import veroviz as vrv
>>> myMap = vrv.addLeafletText(
...     mapObject=None, 
...     mapFilename=None, 
...     mapBackground='CartoDB positron', 
...     mapBoundary=None, 
...     zoomStart=10, 
...     anchorPoint=[43.00154, -78.7871],
...     text="Bell Hall",
...     fontSize=34, 
...     fontColor='black', 
...     horizAlign='left')
>>> myMap

### `createLeaflet()`
This function is used to generate Leaflet objects using folium. The function takes a boundingRegion polygon, `Nodes`, `Arcs`, and `Assignments` dataframes as inputs, and creates a folium/leaflet map showing boundings, nodes and/or arcs.

- See https://veroviz.org/docs/veroviz.createLeaflet.html#veroviz.createLeaflet.createLeaflet for complete function documentation.

In [40]:
# First, import veroviz and check the latest version
>>> import veroviz as vrv
>>> vrv.checkVersion()

'Your current installed version of veroviz is 0.2.3, the latest version available is 0.2.2. To update to the latest version, type `pip install --upgrade veroviz` at a command-line prompt.'

In [41]:
# Now, generate some example nodes inside a bounding region
>>> bounding = [
...     [42.98355351219673, -78.90518188476564],
...     [43.04731443361136, -78.83857727050783],
...     [43.02221961002041, -78.7108612060547],
...     [42.92777124914475, -78.68957519531251],
...     [42.866402688514626, -78.75343322753908],
...     [42.874957707517865, -78.82415771484375],
...     [42.90111863978987, -78.86878967285158],
...     [42.92224052343343, -78.8921356201172]]

>>> exampleNodes = vrv.generateNodes(
...     nodeType         = 'customer', 
...     nodeDistrib      = 'normalBB', 
...     nodeDistribArgs  = {
...         'center' : [42.90, -78.80], 
...         'stdDev' : 10000,
...         'boundingRegion' : bounding
...     },
...     numNodes         = 3,
...     leafletColor = 'orange')

>>> exampleNodes

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,1,42.903444,-78.806811,0.0,None,customer,glyphicon,info-sign,orange,1,pin,Cesium.Color.BLUE,1
1,2,42.931291,-78.803082,0.0,None,customer,glyphicon,info-sign,orange,2,pin,Cesium.Color.BLUE,2
2,3,42.962221,-78.820037,0.0,None,customer,glyphicon,info-sign,orange,3,pin,Cesium.Color.BLUE,3


The first example is using all default values for creating a Leaflet map of the nodes in the `exampleNodes` dataframe.

In [42]:
>>> vrv.createLeaflet(nodes=exampleNodes)

In [43]:
# Define some arcs based on the nodes we just generated:
>>> exampleArcs = vrv.createArcsFromNodeSeq(
...     nodes=exampleNodes,
...     nodeSeq=[1, 2, 3])
>>> exampleArcs

,odID,objectID,startLat,startLon,endLat,endLon,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,None,42.903444,-78.806811,42.931291,-78.803082,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
1,2,None,42.931291,-78.803082,42.962221,-78.820037,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8


In [44]:
# Display the nodes, arcs, and bounding region simultaneously:
>>> vrv.createLeaflet(
...     nodes          = exampleNodes,
...     arcs           = exampleArcs,
...     boundingRegion = bounding)

In [45]:
# The createLeaflet function provides options to override styles
# that were defined in the input nodes and/or arcs dataframes.
# Note:  These overrides will not change the contents in the dataframes.
nodesAndArcsMap = vrv.createLeaflet(nodes      = exampleNodes,
                                    iconPrefix = 'fa',
                                    iconType   = 'car',
                                    iconColor  = 'blue',
                                    arcs       = exampleArcs,
                                    arcStyle   = 'dotted')
nodesAndArcsMap

In [46]:
# If you already have a folium map object, you can add more into it.
# Here, we add a bounding region to the `nodesAndArcsMap` object defined above.
nodesAndArcsMap = vrv.createLeaflet(mapObject      = nodesAndArcsMap,
                                    boundingRegion = bounding)
nodesAndArcsMap

In [47]:
# A new collection of nodes is defined here:
>>> newNodes = vrv.generateNodes(
...     nodeType        = 'customer', 
...     nodeDistrib     = 'uniformBB', 
...     nodeDistribArgs = {
...         'boundingRegion' : bounding
...     },
...     numNodes        = 4,
...     leafletColor    = 'red')
>>> newNodes

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,1,42.936018,-78.844789,0,None,customer,glyphicon,info-sign,red,1,pin,Cesium.Color.BLUE,1
1,2,42.965819,-78.706942,0,None,customer,glyphicon,info-sign,red,2,pin,Cesium.Color.BLUE,2
2,3,42.942236,-78.853390,0,None,customer,glyphicon,info-sign,red,3,pin,Cesium.Color.BLUE,3
3,4,42.929136,-78.744842,0,None,customer,glyphicon,info-sign,red,4,pin,Cesium.Color.BLUE,4


In [48]:
# We will add these nodes to our existing map,
# but we're overriding these new nodes with a green color.
# Notice that the addition of new entities will not change 
# the style of previous entities that were already added into the map.
newMapWithArcsAndMoreNodes = vrv.createLeaflet(
     mapObject = nodesAndArcsMap,
     nodes     = newNodes,
     iconColor = 'green')
newMapWithArcsAndMoreNodes

### All functional arguments

In [49]:
>>> # This example includes all of the available function arguments.
>>> import veroviz as vrv
>>> vrv.createLeaflet(
...     mapObject       = None, 
...     mapFilename     = 'example.html', 
...     mapBackground   = 'CartoDB positron', 
...     mapBoundary     = None, 
...     zoomStart       = 10, 
...     nodes           = exampleNodes, 
...     iconPrefix      = 'fa', 
...     iconType        = 'flag', 
...     iconColor       = 'red', 
...     iconText        = 'Here are some nodes', 
...     arcs            = exampleArcs, 
...     arcWeight       = 5, 
...     arcStyle        = 'dashed', 
...     arcOpacity      = 1, 
...     arcColor        = 'green', 
...     useArrows       = True, 
...     boundingRegion  = bounding, 
...     boundingWeight  = 1, 
...     boundingOpacity = 0.8, 
...     boundingStyle   = 'dotted', 
...     boundingColor   = 'black')

Message: Map page written to example.html.


## generateArcs Module

This module includes the following functions:
- `createArcsFromLocSeq()`
- `createArcsFromNodeSeq()`

### `createArcsFromLocSeq()`
Create an "arcs" dataframe from an ordered list of coordinates.

- See https://veroviz.org/docs/veroviz.generateArcs.html#veroviz.generateArcs.createArcsFromLocSeq for complete function documentation.

In [50]:
# Generate arcs from a given ordered list of coordinates:
>>> import veroviz as vrv
>>> vrv.checkVersion()

>>> arcs = vrv.createArcsFromLocSeq(
...     locSeq=[
...         [42.1325, -78.2134],
...         [42.5341, -78.3252],
...         [42.3424, -78.6424]
...     ])

>>> arcs

,odID,objectID,startLat,startLon,endLat,endLon,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,None,42.1325,-78.2134,42.5341,-78.3252,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
1,2,None,42.5341,-78.3252,42.3424,-78.6424,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8


In [51]:
# Display the arcs on a Leaflet map:
>>> vrv.createLeaflet(arcs=arcs)

In [52]:
>>> # This example includes all of the available function arguments.
>>> arcs = vrv.createArcsFromLocSeq(
...     locSeq         = [[42.1325, -78.2134],
...                       [42.5341, -78.3252],
...                       [42.3424, -78.6424]],
...     initArcs       = None, 
...     startArc       = 1, 
...     objectID       = 'car', 
...     leafletColor   = 'orange', 
...     leafletWeight  = 5, 
...     leafletStyle   = 'dashed', 
...     leafletOpacity = 0.6, 
...     useArrows      = False, 
...     cesiumColor    = 'Cesium.Color.ORANGE', 
...     cesiumWeight   = 5, 
...     cesiumStyle    = 'dashed', 
...     cesiumOpacity  = 0.6)
>>> vrv.createLeaflet(arcs=arcs)

### `createArcsFromNodeSeq()`
Create an "arcs" dataframe from an ordered list of node IDs.  The "nodes" dataframe from which these node IDs are drawn must also be specified.

- See https://veroviz.org/docs/veroviz.generateArcs.html#veroviz.generateArcs.createArcsFromNodeSeq for complete function documentation.

In [53]:
# Import veroviz and check if it is the latest version
>>> import veroviz as vrv
>>> vrv.checkVersion()

'Your current installed version of veroviz is 0.2.3, the latest version available is 0.2.2. To update to the latest version, type `pip install --upgrade veroviz` at a command-line prompt.'

In [54]:
# Create a collection of 20 nodes:
>>> myNodes = vrv.generateNodes(
...     numNodes        = 20,
...     nodeType        = 'depot', 
...     nodeDistrib     = 'normal', 
...     nodeDistribArgs = {
...         'center' : [42.30, 78.00], 
...         'stdDev' : 1000
...     })
>>> myNodes

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,1,42.299068,78.001276,0.0,None,depot,glyphicon,info-sign,blue,1,pin,Cesium.Color.BLUE,1
1,2,42.299054,78.003709,0.0,None,depot,glyphicon,info-sign,blue,2,pin,Cesium.Color.BLUE,2
2,3,42.305833,78.005466,0.0,None,depot,glyphicon,info-sign,blue,3,pin,Cesium.Color.BLUE,3
3,4,42.293216,77.999404,0.0,None,depot,glyphicon,info-sign,blue,4,pin,Cesium.Color.BLUE,4
4,5,42.298142,78.001315,0.0,None,depot,glyphicon,info-sign,blue,5,pin,Cesium.Color.BLUE,5
5,6,42.299681,77.988685,0.0,None,depot,glyphicon,info-sign,blue,6,pin,Cesium.Color.BLUE,6
6,7,42.279575,78.007477,0.0,None,depot,glyphicon,info-sign,blue,7,pin,Cesium.Color.BLUE,7
7,8,42.299976,77.999186,0.0,None,depot,glyphicon,info-sign,blue,8,pin,Cesium.Color.BLUE,8
8,9,42.302103,78.011767,0.0,None,depot,glyphicon,info-sign,blue,9,pin,Cesium.Color.BLUE,9
9,10,42.302286,78.001419,0.0,None,depot,glyphicon,info-sign,blue,10,pin,Cesium.Color.BLUE,10


In [55]:
# Generate arcs from nodes 2-to-15 and from 15-to-8.  
# These node IDs are found in the `id` column of the given Nodes dataframe.
myArcs = vrv.createArcsFromNodeSeq(
            nodeSeq = [2, 15, 8],
            nodes   = myNodes)
myArcs

,odID,objectID,startLat,startLon,endLat,endLon,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,None,42.299054,78.003709,42.299927,78.010292,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
1,2,None,42.299927,78.010292,42.299976,77.999186,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8


In [56]:
# Display the nodes and arcs on a Leaflet map:
>>> myMap = vrv.createLeaflet(arcs=myArcs, nodes=myNodes)
>>> myMap

In [57]:
# This example includes all of the available function arguments:
>>> moreArcs = vrv.createArcsFromNodeSeq(
...     nodeSeq        = [3, 16, 9],
...     nodes          = myNodes, 
...     initArcs       = myArcs, 
...     startArc       = 7, 
...     objectID       = 'car',
...     leafletColor   = 'cadetblue', 
...     leafletWeight  = 3, 
...     leafletStyle   = 'dotted', 
...     leafletOpacity = 0.8, 
...     useArrows      = False, 
...     cesiumColor    = 'Cesium.Color.CADETBLUE', 
...     cesiumWeight   = 3, 
...     cesiumStyle    = 'dotted', 
...     cesiumOpacity  = 0.8)
>>> moreArcs

,odID,objectID,startLat,startLon,endLat,endLon,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,None,42.299054,78.003709,42.299927,78.010292,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
1,2,None,42.299927,78.010292,42.299976,77.999186,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
2,7,car,42.305833,78.005466,42.298705,77.998409,cadetblue,3,dotted,0.8,False,Cesium.Color.CADETBLUE,3,dotted,0.8
3,8,car,42.298705,77.998409,42.302103,78.011767,cadetblue,3,dotted,0.8,False,Cesium.Color.CADETBLUE,3,dotted,0.8


In [58]:
# Display the nodes and arcs on a Leaflet map:
>>> vrv.createLeaflet(mapObject=myMap, arcs=moreArcs)

---
## generateNodes Module

This module includes the following functions:
- `generateNodes()`
- `createNodesFromLocs()`

### `generateNodes()`
This function generates a collection of nodes (locations).

- See https://veroviz.org/docs/veroviz.generateNodes.html#veroviz.generateNodes.generateNodes for complete function documentation.

In [41]:
# Import veroviz and check if it is the latest version:
>>> import veroviz as vrv
>>> vrv.checkVersion()

'Your current installed version of veroviz is 0.2.0. You are up-to-date with the latest available version.'

In [42]:
# This first example will generate 20 nodes, normally distributed. 
# The distribution is centered at lat 42.30, lon 78.00. 
# The distribution has a standard deviation of 1000 meters.
myNodes = vrv.generateNodes(
    numNodes        = 20,
    nodeType        = 'depot', 
    nodeDistrib     = 'normal', 
    nodeDistribArgs = {
        'center' : [42.30, -78.00], 
        'stdDev' : 1000
    })
myNodes

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,1,42.299758,-78.000274,0.0,None,depot,glyphicon,info-sign,blue,1,pin,Cesium.Color.BLUE,1
1,2,42.302888,-78.014779,0.0,None,depot,glyphicon,info-sign,blue,2,pin,Cesium.Color.BLUE,2
2,3,42.297674,-78.004709,0.0,None,depot,glyphicon,info-sign,blue,3,pin,Cesium.Color.BLUE,3
3,4,42.300086,-77.999806,0.0,None,depot,glyphicon,info-sign,blue,4,pin,Cesium.Color.BLUE,4
4,5,42.300584,-77.999657,0.0,None,depot,glyphicon,info-sign,blue,5,pin,Cesium.Color.BLUE,5
5,6,42.300472,-78.011399,0.0,None,depot,glyphicon,info-sign,blue,6,pin,Cesium.Color.BLUE,6
6,7,42.302801,-77.994909,0.0,None,depot,glyphicon,info-sign,blue,7,pin,Cesium.Color.BLUE,7
7,8,42.298251,-78.003272,0.0,None,depot,glyphicon,info-sign,blue,8,pin,Cesium.Color.BLUE,8
8,9,42.299522,-78.000697,0.0,None,depot,glyphicon,info-sign,blue,9,pin,Cesium.Color.BLUE,9
9,10,42.299938,-78.000007,0.0,None,depot,glyphicon,info-sign,blue,10,pin,Cesium.Color.BLUE,10


In [43]:
# View the center point, 1 std dev, 3 std devs, and resulting nodes on a Leaflet map:
>>> myMap = vrv.addLeafletMarker(center      = [42.30, -78.00], 
...                              fillOpacity = 1)
>>> myMap = vrv.addLeafletCircle(mapObject = myMap, 
...                              center    = [42.30, -78.00], 
...                              radius    = 1000, 
...                              fillColor = 'green')
>>> myMap = vrv.addLeafletCircle(mapObject = myMap, 
...                              center    = [42.30, -78.00], 
...                              radius    = 3*1000)
>>> myMap = vrv.createLeaflet(mapObject = myMap, 
...                           nodes     = myNodes)
>>> myMap

In [44]:
# The following examples require a bounding region. For example:
>>> bounding = [
...     [42.98355351219673, -78.90518188476564], 
...     [43.04731443361136, -78.83857727050783], 
...     [43.02221961002041, -78.7108612060547], 
...     [42.92777124914475, -78.68957519531251], 
...     [42.866402688514626, -78.75343322753908], 
...     [42.874957707517865, -78.82415771484375], 
...     [42.90111863978987, -78.86878967285158], 
...     [42.92224052343343, -78.8921356201172]]

In [45]:
# The second example will give us 20 nodes, normally-distributed, 
# centered at [42.30, 78.00], with a standard deviation of 4000 meters 
# about the center.  However, the nodes must also fall within the bounding region.
myNodes2 = vrv.generateNodes(
    numNodes        = 200,
    nodeType        = 'depot', 
    nodeDistrib     = 'normal', 
    nodeDistribArgs = {
        'center' : [42.90, -78.80], 
        'stdDev' : 4000,
        'boundingRegion' : bounding
    })

myNodes2

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,1,42.904277,-78.793387,0.0,None,depot,glyphicon,info-sign,blue,1,pin,Cesium.Color.BLUE,1
1,2,42.868993,-78.770704,0.0,None,depot,glyphicon,info-sign,blue,2,pin,Cesium.Color.BLUE,2
2,3,42.898596,-78.815039,0.0,None,depot,glyphicon,info-sign,blue,3,pin,Cesium.Color.BLUE,3
3,4,42.933351,-78.796172,0.0,None,depot,glyphicon,info-sign,blue,4,pin,Cesium.Color.BLUE,4
4,5,42.886314,-78.800701,0.0,None,depot,glyphicon,info-sign,blue,5,pin,Cesium.Color.BLUE,5
5,6,42.928992,-78.770664,0.0,None,depot,glyphicon,info-sign,blue,6,pin,Cesium.Color.BLUE,6
6,7,42.966101,-78.859476,0.0,None,depot,glyphicon,info-sign,blue,7,pin,Cesium.Color.BLUE,7
7,8,42.919606,-78.803901,0.0,None,depot,glyphicon,info-sign,blue,8,pin,Cesium.Color.BLUE,8
8,9,42.896307,-78.764590,0.0,None,depot,glyphicon,info-sign,blue,9,pin,Cesium.Color.BLUE,9
9,10,42.902922,-78.790991,0.0,None,depot,glyphicon,info-sign,blue,10,pin,Cesium.Color.BLUE,10


In [47]:
# View the center point, 1 std dev, 3 std devs, bounding region, and resulting nodes on a Leaflet map:
>>> myMap2 = vrv.addLeafletMarker(center      = [42.90, -78.80], 
...                               fillOpacity = 1)
>>> myMap2 = vrv.addLeafletCircle(mapObject = myMap2, 
...                               center    = [42.90, -78.80], 
...                               radius    = 4000, 
...                               fillColor = 'green')
>>> myMap2 = vrv.addLeafletCircle(mapObject = myMap2, 
...                               center    = [42.90, -78.80], 
...                               radius    = 3*4000)
>>> myMap2 = vrv.createLeaflet(mapObject      = myMap2, 
...                            nodes          = myNodes2,
...                            boundingRegion = bounding)
>>> myMap2

In [48]:
# The third example will generate 20 nodes uniformly distributed in a given bounding region:
>>> myNodes3 = vrv.generateNodes(
...     numNodes        = 20, 
...     nodeDistrib     = 'uniformBB', 
...     nodeDistribArgs = {
...         'boundingRegion' : bounding
...     })
>>> myNodes3

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,1,43.006914,-78.836186,0,None,None,glyphicon,info-sign,blue,1,pin,Cesium.Color.BLUE,1
1,2,42.977760,-78.725205,0,None,None,glyphicon,info-sign,blue,2,pin,Cesium.Color.BLUE,2
2,3,42.980416,-78.890527,0,None,None,glyphicon,info-sign,blue,3,pin,Cesium.Color.BLUE,3
3,4,43.035625,-78.822513,0,None,None,glyphicon,info-sign,blue,4,pin,Cesium.Color.BLUE,4
4,5,42.938563,-78.894011,0,None,None,glyphicon,info-sign,blue,5,pin,Cesium.Color.BLUE,5
5,6,42.978255,-78.710089,0,None,None,glyphicon,info-sign,blue,6,pin,Cesium.Color.BLUE,6
6,7,42.883315,-78.823439,0,None,None,glyphicon,info-sign,blue,7,pin,Cesium.Color.BLUE,7
7,8,42.927722,-78.776708,0,None,None,glyphicon,info-sign,blue,8,pin,Cesium.Color.BLUE,8
8,9,42.923780,-78.823737,0,None,None,glyphicon,info-sign,blue,9,pin,Cesium.Color.BLUE,9
9,10,42.959813,-78.870510,0,None,None,glyphicon,info-sign,blue,10,pin,Cesium.Color.BLUE,10


In [49]:
# View the bounding region and generated nodes on a Leaflet map:
>>> myMap3 = vrv.createLeaflet(nodes          = myNodes3,
...                            boundingRegion = bounding)
>>> myMap3

In [50]:
# The final example includes all available function arguments:
>>> myNodes4 = vrv.generateNodes(
...     initNodes         = None,
...     nodeType          = 'warehouse',
...     nodeName          = 'WH ',  
...     numNodes          = 5,
...     startNode         = 101,
...     incrementName     = True,
...     incrementStart    = 1,
...     nodeDistrib       = 'uniformBB',
...     nodeDistribArgs   = {
...         'boundingRegion' : bounding
...     },
...     snapToRoad        = True,
...     leafletIconPrefix = 'fa',
...     leafletIconType   = 'star',
...     leafletColor      = 'darkred',
...     leafletIconText   = 'These nodes are used for demo',
...     cesiumIconType    = 'pin',
...     cesiumColor       = 'Cesium.Color.DARKRED',
...     cesiumIconText    = None,
...     dataProvider      = 'OSRM-online',
...     dataProviderArgs  = None)

>>> myNodes4

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,101,42.946013,-78.711883,0,WH 1,warehouse,fa,star,darkred,These nodes are used for demo,pin,Cesium.Color.DARKRED,101
1,102,43.022784,-78.810654,0,WH 2,warehouse,fa,star,darkred,These nodes are used for demo,pin,Cesium.Color.DARKRED,102
2,103,42.977206,-78.781993,0,WH 3,warehouse,fa,star,darkred,These nodes are used for demo,pin,Cesium.Color.DARKRED,103
3,104,42.956513,-78.793406,0,WH 4,warehouse,fa,star,darkred,These nodes are used for demo,pin,Cesium.Color.DARKRED,104
4,105,42.898479,-78.766509,0,WH 5,warehouse,fa,star,darkred,These nodes are used for demo,pin,Cesium.Color.DARKRED,105


In [51]:
# View the bounding region and generated nodes on a Leaflet map:
>>> myMap4 = vrv.createLeaflet(nodes          = myNodes4,
...                            boundingRegion = bounding)
>>> myMap4

### `createNodesFromLocs()`
This function generates a "nodes" dataframe from a given collection of [lat, lon], or [lat, lon, alt],  coordinates. 

- See https://veroviz.org/docs/veroviz.generateNodes.html#veroviz.generateNodes.createNodesFromLocs for complete function documentation.

In [52]:
# Import veroviz and check if it is the latest version:
>>> import veroviz as vrv
>>> vrv.checkVersion()

'Your current installed version of veroviz is 0.2.0. You are up-to-date with the latest available version.'

In [53]:
# Generate nodes from a list of [lat, lon] pairs (no altitude specified):
>>> nodes2D = vrv.createNodesFromLocs(
...     locs=[
...         [42.1538, -78.4253],
...         [42.3465, -78.6234],
...         [42.6343, -78.1146]])
>>> nodes2D

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,1,42.1538,-78.4253,0,None,None,glyphicon,info-sign,blue,1,pin,Cesium.Color.BLUE,1
1,2,42.3465,-78.6234,0,None,None,glyphicon,info-sign,blue,2,pin,Cesium.Color.BLUE,2
2,3,42.6343,-78.1146,0,None,None,glyphicon,info-sign,blue,3,pin,Cesium.Color.BLUE,3


In [54]:
# Generate nodes from a list of [lat, lon, alt] pairs:
>>> nodes3D = vrv.createNodesFromLocs(
...     locs=[
...         [42.1538, -78.4253, 200],
...         [42.3465, -78.6234, 400],
...         [42.6343, -78.1146, 200]])
>>> nodes3D

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,1,42.1538,-78.4253,200,None,None,glyphicon,info-sign,blue,1,pin,Cesium.Color.BLUE,1
1,2,42.3465,-78.6234,400,None,None,glyphicon,info-sign,blue,2,pin,Cesium.Color.BLUE,2
2,3,42.6343,-78.1146,200,None,None,glyphicon,info-sign,blue,3,pin,Cesium.Color.BLUE,3


In [55]:
# This example includes all function arguments:
>>> myLocs = [[42.1538, -78.4253],
...           [42.3465, -78.6234],
...           [42.6343, -78.1146]]

>>> myNodes = vrv.createNodesFromLocs(
...     locs              = myLocs, 
...     initNodes         = None, 
...     nodeType          = 'customers', 
...     nodeName          = 'cust', 
...     startNode         = 1, 
...     incrementName     = True, 
...     incrementStart    = 7, 
...     snapToRoad        = False, 
...     dataProvider      = None, 
...     dataProviderArgs  = None,
...     leafletIconPrefix = 'fa', 
...     leafletIconType   = 'user', 
...     leafletColor      = 'lightgreen', 
...     leafletIconText   = None, 
...     cesiumIconType    = 'pin', 
...     cesiumColor       = 'Cesium.Color.LIGHTGREEN', 
...     cesiumIconText    = None)
>>> myNodes

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,1,42.1538,-78.4253,0,cust7,customers,fa,user,lightgreen,1,pin,Cesium.Color.LIGHTGREEN,1
1,2,42.3465,-78.6234,0,cust8,customers,fa,user,lightgreen,2,pin,Cesium.Color.LIGHTGREEN,2
2,3,42.6343,-78.1146,0,cust9,customers,fa,user,lightgreen,3,pin,Cesium.Color.LIGHTGREEN,3


In [56]:
# Display the nodes on a Leaflet map:
>>> vrv.createLeaflet(nodes = myNodes)

## getShapepoints2D Module

This module contains the following function:
- `getShapepoints2D()`

### `getShapepoints2D()`
This function generates all of the "shapepoints" between two given GPS coordinates, including timestamps indicating the departure and arrival times for each shapepoint. Shapepoints are pairs of GPS coordinates that are connected by  straight lines.  For a given origin and destination, numerous individual shapepoints can be combined to define a travel route along a road network.

- See https://veroviz.org/docs/veroviz.getShapepoints2D.html#veroviz.getShapepoints2D.getShapepoints2D for complete function documentation.

In [59]:
# Import veroviz and check if it's the latest version:
>>> import veroviz as vrv
>>> vrv.checkVersion()

'Your current installed version of veroviz is 0.2.3, the latest version available is 0.2.2. To update to the latest version, type `pip install --upgrade veroviz` at a command-line prompt.'

In [60]:
# The following examples assume the use of ORS as the data provider.
# If you have saved your API key as an environment variable,
# you may use `os.environ` to access it:
import os
ORS_API_KEY = os.environ['ORSKEY']

# Otherwise, you may specify your key here:
# ORS_API_KEY = 'YOUR_ORS_KEY_GOES_HERE'

In [61]:
# Example 1 - A minimal example, using the fastest car route.  
# The ORS-online data provider requires an API key.
shapepoints2D = vrv.getShapepoints2D(
     odID             = 1,
     startLoc         = [42.80, -78.80],
     endLoc           = [42.90, -78.80],
     routeType        = 'fastest',
     dataProvider     = 'ORS-online',
     dataProviderArgs = {'APIkey': ORS_API_KEY})
shapepoints2D


Message: The origin point (lat: 42.8, lon: -78.8) is 266.2 meters away from the road. You might find a gap between the origin point and the route.
Message: The destination point (lat: 42.9, lon: -78.8) is 145.1 meters away from the road. You might find a gap between destination point and the route.


,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,None,None,100,75,0.000000,42.801336,-78.802702,0,0.000000,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
1,1,None,None,100,75,0.000000,42.801336,-78.802702,0,6.478416,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
2,1,None,None,100,75,6.478416,42.801209,-78.803107,0,12.110697,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
3,1,None,None,100,75,12.110697,42.801465,-78.803267,0,24.129894,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
4,1,None,None,100,75,24.129894,42.801217,-78.804011,0,44.644651,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
5,1,None,None,100,75,44.644651,42.802007,-78.804901,0,48.193520,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
6,1,None,None,100,75,48.193520,42.802176,-78.804975,0,52.763014,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
7,1,None,None,100,75,52.763014,42.802402,-78.805022,0,74.157847,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
8,1,None,None,100,75,74.157847,42.803472,-78.804978,0,95.500000,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
9,1,None,None,100,75,95.500000,42.804539,-78.804920,0,95.682040,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8


In [62]:
# View the route in Leaflet.  The green marker is the start, the red marker is the end:
>>> myMap = vrv.createLeaflet(arcs=shapepoints2D)
>>> myMap = vrv.addLeafletMarker(mapObject=myMap, center=[42.80, -78.80], fillColor='green')
>>> myMap = vrv.addLeafletMarker(mapObject=myMap, center=[42.90, -78.80], fillColor='red')
>>> myMap

In [63]:
# Example 2 - Shapepoints with Euclidean travel can also be generated:
shapepoints2D = vrv.getShapepoints2D(
     odID             = 1,
     startLoc         = [42.80, -78.80],
     endLoc           = [42.90, -78.80],
     routeType        = 'euclidean2D',
     speedMPS         = vrv.convertSpeed(55, 'mi', 'hr', 'm', 's'),
     dataProvider     = None,
     dataProviderArgs = None)
shapepoints2D

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,None,None,100,75,0.0,42.8,-78.8,0,451.820827,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8


In [64]:
# View the route in Leaflet.  The green marker is the start, the red marker is the end:
>>> myMap = vrv.createLeaflet(arcs=shapepoints2D)
>>> myMap = vrv.addLeafletMarker(mapObject=myMap, center=[42.80, -78.80], fillColor='green')
>>> myMap = vrv.addLeafletMarker(mapObject=myMap, center=[42.90, -78.80], fillColor='red')
>>> myMap

In [67]:
# Example 3 - Generate an Assignments dataframe that starts from 520 seconds and 
# assumes the vehicle is driving at a constant constant speed of 16 m/s (or 35.8 mph).  
# The OSRM-online data provider does not require an API key.
shapepoints2D = vrv.getShapepoints2D(
     odID         = 1, 
     startLoc     = [42.80, -78.80], 
     endLoc       = [42.90, -78.80], 
     startTimeSec = 520,
     routeType    = 'fastest',
     dataProvider = 'OSRM-online',
     speedMPS     = 16)
shapepoints2D


Message: The origin point (lat: 42.8, lon: -78.8) is 266.2 meters away from the road. You might find a gap between the origin point and the route.
Message: The destination point (lat: 42.9, lon: -78.8) is 144.9 meters away from the road. You might find a gap between destination point and the route.


,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,None,None,100,75,520,42.801336,-78.802702,0,523.024338,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
1,1,None,None,100,75,523.024,42.801465,-78.803267,0,545.986015,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
2,1,None,None,100,75,545.986,42.804540,-78.804920,0,585.163466,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
3,1,None,None,100,75,585.163,42.804378,-78.797260,0,591.965711,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
4,1,None,None,100,75,591.966,42.805334,-78.797551,0,597.975689,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
5,1,None,None,100,75,597.976,42.806179,-78.797806,0,598.879718,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
6,1,None,None,100,75,598.88,42.806306,-78.797845,0,606.775620,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
7,1,None,None,100,75,606.776,42.807416,-78.798181,0,612.880363,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
8,1,None,None,100,75,612.88,42.808274,-78.798442,0,613.030495,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
9,1,None,None,100,75,613.03,42.808295,-78.798449,0,617.846357,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8


In [68]:
# View the route in Leaflet.  The green marker is the start, the red marker is the end:
>>> myMap = vrv.createLeaflet(arcs=shapepoints2D)
>>> myMap = vrv.addLeafletMarker(mapObject=myMap, center=[42.80, -78.80], fillColor='green')
>>> myMap = vrv.addLeafletMarker(mapObject=myMap, center=[42.90, -78.80], fillColor='red')
>>> myMap

In [69]:
# Example 4 - The `expDurationSec` argument ensures that the vehicle reaches 
# the ending location at time `startTimeSec + expDurationSec`. This is useful 
# when you are using data from different sources (e.g., time matrix data from 
# pgRouting and shapepoints created by ORS) and you want to maintain consistency 
# in timing. If `expDurationSec` and `speedMPS` are both provided, 
# `expDurationSec` will override the `speedMPS`.
[travelTimeSec, travelDistMeters]  = vrv.getTimeDistScalar2D(
     startLoc         = [42.80, -78.80],
     endLoc           = [42.90, -78.80],
     outputDistUnits  = 'meters',
     outputTimeUnits  = 'seconds',
     routeType        = 'fastest',
     dataProvider     = 'ORS-online',
     dataProviderArgs = {'APIkey' : ORS_API_KEY})
shapepoints2D = vrv.getShapepoints2D(
     odID             = 1, 
     startLoc         = [42.80, -78.80], 
     endLoc           = [42.90, -78.80], 
     startTimeSec     = 520,
     routeType        = 'fastest',
     dataProvider     = 'ORS-online',
     dataProviderArgs = {'APIkey' : ORS_API_KEY},
     expDurationSec   = travelTimeSec)
shapepoints2D


Message: The origin point (lat: 42.8, lon: -78.8) is 266.2 meters away from the road. You might find a gap between the origin point and the route.
Message: The destination point (lat: 42.9, lon: -78.8) is 145.1 meters away from the road. You might find a gap between destination point and the route.


,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,None,None,100,75,520,42.801336,-78.802702,0,520.000000,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
1,1,None,None,100,75,520,42.801336,-78.802702,0,522.941209,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
2,1,None,None,100,75,522.941,42.801209,-78.803107,0,525.498272,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
3,1,None,None,100,75,525.498,42.801465,-78.803267,0,530.955001,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
4,1,None,None,100,75,530.955,42.801217,-78.804011,0,540.268727,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
5,1,None,None,100,75,540.269,42.802007,-78.804901,0,541.879918,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
6,1,None,None,100,75,541.88,42.802176,-78.804975,0,543.954473,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
7,1,None,None,100,75,543.954,42.802402,-78.805022,0,553.667754,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
8,1,None,None,100,75,553.668,42.803472,-78.804978,0,563.357118,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
9,1,None,None,100,75,563.357,42.804539,-78.804920,0,563.564238,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8


In [70]:
# Example 5 - Note that `getShapepoints2D()` sometimes displays messages like 
# "The origin/destination point is XXX meters away from the road".  
# This occurs if the start and/or end location provided is too far away from 
# the nearest road.  VeRoViz does not attempt to find a path from an arbitrary l
# ocation to the nearest road.  To avoid these warnings, one option is to snap 
# the start/end locations to the road network before attempting to get the 
# shapepoints.  In this case, it is highly recommended to use the same data provider
# for both the snapping and shapepoint activities.
startLoc = vrv.getSnapLoc(loc              = [42.80, -78.80], 
                          dataProvider     = 'ORS-online',
                          dataProviderArgs = {'APIkey': ORS_API_KEY})
endLoc   = vrv.getSnapLoc(loc              = [42.90, -78.80], 
                          dataProvider     = 'ORS-online',
                          dataProviderArgs = {'APIkey': ORS_API_KEY})

shapepoints2D = vrv.getShapepoints2D(
     odID             = 1, 
     startLoc         = startLoc,
     endLoc           = endLoc,
     startTimeSec     = 520,
     routeType        = 'fastest',
     dataProvider     = 'ORS-online',
     dataProviderArgs = {'APIkey': ORS_API_KEY})

shapepoints2D

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,None,None,100,75,520,42.801336,-78.802702,0,520.000000,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
1,1,None,None,100,75,520,42.801336,-78.802702,0,526.478416,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
2,1,None,None,100,75,526.478,42.801209,-78.803107,0,532.110697,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
3,1,None,None,100,75,532.111,42.801465,-78.803267,0,544.129894,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
4,1,None,None,100,75,544.13,42.801217,-78.804011,0,564.644651,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
5,1,None,None,100,75,564.645,42.802007,-78.804901,0,568.193520,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
6,1,None,None,100,75,568.194,42.802176,-78.804975,0,572.763014,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
7,1,None,None,100,75,572.763,42.802402,-78.805022,0,594.157847,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
8,1,None,None,100,75,594.158,42.803472,-78.804978,0,615.500000,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
9,1,None,None,100,75,615.5,42.804539,-78.804920,0,615.682040,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8


In [71]:
# View the route in Leaflet.  
# The green filled marker is the original start, 
# the green outlined marker is the snapped start.
# The red filled marker is the original end,
# the red outlined marker is the snapped end.
myMap = vrv.createLeaflet(arcs=shapepoints2D)
myMap = vrv.addLeafletMarker(mapObject=myMap, center=[42.80, -78.80], 
                             fillColor='green')
myMap = vrv.addLeafletMarker(mapObject=myMap, center=[42.90, -78.80], 
                             fillColor='red')
myMap = vrv.addLeafletMarker(mapObject=myMap, center=startLoc, 
                             fillColor=None, lineColor='green')
myMap = vrv.addLeafletMarker(mapObject=myMap, center=endLoc, 
                             fillColor=None, lineColor='red')
myMap

In [76]:
# Example 6 - If you are constructing the Assignments dataframe for use 
# with Cesium, then the `objectID` and `modelFile` arguments are required. 
# If you are just using it to generate Leaflet maps, those arguments can 
# be left as default (None).
shapepoints2D = vrv.getShapepoints2D(
    odID             = 1, 
    objectID         = 'Blue Car',
    modelFile        = '/veroviz/models/car_blue.gltf',
    startLoc         = vrv.getSnapLoc(
                         loc          = [42.80, -78.80], 
                         dataProvider = 'OSRM-online'),
    endLoc           = vrv.getSnapLoc(
                         loc          = [42.90, -78.80], 
                         dataProvider = 'OSRM-online'),
    startTimeSec     = 520,
    routeType        = 'shortest',
    dataProvider     = 'MapQuest',
    dataProviderArgs = {'APIkey': os.environ['MAPQUESTKEY']},
    expDurationSec   = 1500)
shapepoints2D


Message: The origin point (lat: 42.801336, lon: -78.802702) is 94.0 meters away from the road. You might find a gap between the origin point and the route.
Message: The destination point (lat: 42.898714, lon: -78.7997) is 14.1 meters away from the road. You might find a gap between destination point and the route.


,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,520,42.801113,-78.803810,0,522.124430,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
1,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,522.124,42.801224,-78.803986,0,526.786827,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
2,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,526.787,42.801495,-78.804337,0,530.256110,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
3,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,530.256,42.801708,-78.804581,0,533.094421,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
4,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,533.094,42.801888,-78.804771,0,535.217293,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
5,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,535.217,42.802036,-78.804886,0,537.292381,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
6,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,537.292,42.802193,-78.804962,0,539.033220,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
7,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,539.033,42.802330,-78.805000,0,554.611844,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
8,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,554.612,42.803581,-78.804962,0,566.402647,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
9,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,566.403,42.804527,-78.804901,0,575.638536,...,0,orange,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8


In [77]:
# Example 7 - This example includes all of the functional arguments:
>>> start = [42.80, -78.80]
>>> end   = [42.90, -78.80]

>>> shapepoints2D = vrv.getShapepoints2D(
...     odID             = 1, 
...     objectID         = 'Blue Car', 
...     modelFile        = 'veroviz/models/car_blue.gltf', 
...     modelScale       = 100,
...     modelMinPxSize   = 75,
...     startLoc         = start, 
...     endLoc           = end, 
...     startTimeSec     = 30.0, 
...     expDurationSec   = 90, 
...     routeType        = 'shortest', 
...     speedMPS         = 5.2, 
...     leafletColor     = 'blue', 
...     leafletWeight    = 3, 
...     leafletStyle     = 'dashed', 
...     leafletOpacity   = 0.8, 
...     useArrows        = True, 
...     cesiumColor      = 'Cesium.Color.BLUE', 
...     cesiumWeight     = 3, 
...     cesiumStyle      = 'solid', 
...     cesiumOpacity    = 0.8, 
...     dataProvider     = 'MapQuest',
...     dataProviderArgs = {'APIkey': os.environ['MAPQUESTKEY']})
 
>>> myMap = vrv.createLeaflet(arcs = shapepoints2D)
>>> myMap = vrv.addLeafletMarker(mapObject=myMap, center=start, fillColor='green', lineColor='green')
>>> myMap = vrv.addLeafletMarker(mapObject=myMap, center=end, fillColor='red', lineColor='red')
>>> myMap


Message: The origin point (lat: 42.8, lon: -78.8) is 332.2 meters away from the road. You might find a gap between the origin point and the route.
Message: The destination point (lat: 42.9, lon: -78.8) is 141.1 meters away from the road. You might find a gap between destination point and the route.


In [70]:
# Generate a Cesium movie:
>>> vrv.createCesium(
...     assignments = shapepoints2D, 
...     cesiumDir   = os.environ['CESIUMDIR'],
...     problemDir  = '/examples/shapepoints')

Message: File selector was written to /Users/murray/cesium/examples/shapepoints/;examples;shapepoints.vrv ...
Message: Configs were written to /Users/murray/cesium/examples/shapepoints/config.js ...
Message: Assignments (.js) were written to /Users/murray/cesium/examples/shapepoints/displayPaths.js ...
Message: Assignments (.czml) were written to /Users/murray/cesium/examples/shapepoints/routes.czml ...


## getShapepoints3D Module
This module contains the following function:
- `getShapepoints3D()`

### `getShapepoints3D()`
This function generates 3-dimensional "shapepoints" between two given GPS coordinates, including timestamps indicating the departure and arrival times for each shapepoint. Shapepoints are pairs of GPS coordinates (and altitudes) that are connected by  straight lines.  For a given origin and destination, numerous individual shapepoints can be combined to define movement in three dimensions.

- See https://veroviz.org/docs/veroviz.getShapepoints3D.html#module-veroviz.getShapepoints3D for complete function documentation.

In [71]:
# Import veroviz and check if a newer version exists:
>>> import veroviz as vrv
>>> vrv.checkVersion()

'Your current installed version of veroviz is 0.2.0. You are up-to-date with the latest available version.'

In [72]:
# Example 1 - Square profile (takeoff and land vertically, like a quadcopter).
>>> shapepoints3D_1 = vrv.getShapepoints3D(
...     odID               = 1,
...     objectID           = 'square flight', 
...     modelFile          = '/veroviz/models/drone.gltf',
...     startLoc           = [42.80, -78.80],
...     endLoc             = [42.80, -78.77],
...     cruiseAltMetersAGL = 220,
...     takeoffSpeedMPS    = 15,
...     cruiseSpeedMPS     = 20,
...     landSpeedMPS       = 10,
...     climbRateMPS       = None,
...     descentRateMPS     = None,    
...     routeType          = 'square',
...     cesiumColor        = 'Cesium.Color.RED')
>>> shapepoints3D_1

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,square flight,/veroviz/models/drone.gltf,100,75,0.000000,42.8,-78.80,0,14.666667,...,220,orange,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
1,1,square flight,/veroviz/models/drone.gltf,100,75,14.666667,42.8,-78.80,220,137.374072,...,220,orange,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
2,1,square flight,/veroviz/models/drone.gltf,100,75,137.374072,42.8,-78.77,220,159.374072,...,0,orange,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8


In [73]:
# Example 2 - Straight profile (fly directly from start to end).
>>> shapepoints3D_2 = vrv.getShapepoints3D(
...     odID               = 2,
...     objectID           = 'straight flight', 
...     modelFile          = '/veroviz/models/drone.gltf',
...     startLoc           = [42.80, -78.80],
...     endLoc             = [42.80, -78.77, 219],
...     cruiseAltMetersAGL = None, 
...     takeoffSpeedMPS    = None, 
...     cruiseSpeedMPS     = 20,
...     landSpeedMPS       = None, 
...     climbRateMPS       = None,
...     descentRateMPS     = None,    
...     routeType          = 'straight',
...     cesiumColor        = 'Cesium.Color.GREEN')
>>> shapepoints3D_2

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,2,straight flight,/veroviz/models/drone.gltf,100,75,0.0,42.8,-78.8,0,123.195008,...,219,orange,3,solid,0.8,True,Cesium.Color.GREEN,3,solid,0.8


In [74]:
# Example 3 - Trapezoidal profiles (with different climb/descent rates).
>>> shapepoints3D_3a = vrv.getShapepoints3D(
...     odID               = 3,
...     objectID           = 'trapezoid a', 
...     modelFile          = '/veroviz/models/drone.gltf',
...     startLoc           = [42.80, -78.80],
...     endLoc             = [42.80, -78.77],
...     cruiseAltMetersAGL = 220,
...     takeoffSpeedMPS    = 15,
...     cruiseSpeedMPS     = 20,
...     landSpeedMPS       = 10,
...     climbRateMPS       = 10,
...     descentRateMPS     = 5,    
...     routeType          = 'trapezoidal',
...     cesiumColor        = 'Cesium.Color.BLACK')
>>> shapepoints3D_3a

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,3,trapezoid a,/veroviz/models/drone.gltf,100,75,0.000000,42.8,-78.800000,0,22.000000,...,220,orange,3,solid,0.8,True,Cesium.Color.BLACK,3,solid,0.8
1,3,trapezoid a,/veroviz/models/drone.gltf,100,75,22.000000,42.8,-78.796993,220,113.356473,...,220,orange,3,solid,0.8,True,Cesium.Color.BLACK,3,solid,0.8
2,3,trapezoid a,/veroviz/models/drone.gltf,100,75,113.356473,42.8,-78.774658,220,157.356473,...,0,orange,3,solid,0.8,True,Cesium.Color.BLACK,3,solid,0.8


In [75]:
>>> shapepoints3D_3b = vrv.getShapepoints3D(
...     odID               = 4,
...     objectID           = 'trapezoid b', 
...     modelFile          = '/veroviz/models/drone.gltf',
...     startLoc           = [42.80, -78.80],
...     endLoc             = [42.80, -78.77],
...     cruiseAltMetersAGL = 220,
...     takeoffSpeedMPS    = 15,
...     cruiseSpeedMPS     = 20,
...     landSpeedMPS       = 10,
...     climbRateMPS       = 5,
...     descentRateMPS     = 1,    
...     routeType          = 'trapezoidal',
...     cesiumColor        = 'Cesium.Color.BLACK',
...     cesiumStyle        = 'dashed')
>>> shapepoints3D_3b

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,4,trapezoid b,/veroviz/models/drone.gltf,100,75,0.000000,42.8,-78.80,0,245.414811,...,0,orange,3,solid,0.8,True,Cesium.Color.BLACK,3,dashed,0.8
1,4,trapezoid b,/veroviz/models/drone.gltf,100,75,245.414811,42.8,-78.77,0,319.703048,...,1114.32,orange,3,solid,0.8,True,Cesium.Color.BLACK,3,dashed,0.8


In [76]:
# Example 4 - Triangular profile
>>> shapepoints3D_4 = vrv.getShapepoints3D(
...     odID               = 5,
...     objectID           = 'triangle', 
...     modelFile          = '/veroviz/models/drone.gltf',
...     startLoc           = [42.80, -78.80],
...     endLoc             = [42.80, -78.77],
...     cruiseAltMetersAGL = 220,
...     takeoffSpeedMPS    = None,
...     cruiseSpeedMPS     = 20,
...     landSpeedMPS       = None,
...     climbRateMPS       = None,
...     descentRateMPS     = None,    
...     routeType          = 'triangular',
...     cesiumColor        = 'Cesium.Color.LIGHTPINK')
>>> shapepoints3D_4

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,5,triangle,/veroviz/models/drone.gltf,100,75,0.000000,42.8,-78.800,0,62.331989,...,220,orange,3,solid,0.8,True,Cesium.Color.LIGHTPINK,3,solid,0.8
1,5,triangle,/veroviz/models/drone.gltf,100,75,62.331989,42.8,-78.785,220,186.995968,...,0,orange,3,solid,0.8,True,Cesium.Color.LIGHTPINK,3,solid,0.8


In [77]:
# Example 5 - Compare the profiles from Examples 1-4.

# We'll need pandas to concatenate all of our dataframes.
import pandas as pd

# Initialize an empty "assignments" dataframe:
assignmentsDF = vrv.initDataframe('assignments')

# Append to the assignments dataframe.
# The assignments dataframe will hold all shapepoints for all vehicles.
assignmentsDF = pd.concat([assignmentsDF, shapepoints3D_1], ignore_index=True, sort=False)
assignmentsDF = pd.concat([assignmentsDF, shapepoints3D_2], ignore_index=True, sort=False)
assignmentsDF = pd.concat([assignmentsDF, shapepoints3D_3a], ignore_index=True, sort=False)
assignmentsDF = pd.concat([assignmentsDF, shapepoints3D_3b], ignore_index=True, sort=False)
assignmentsDF = pd.concat([assignmentsDF, shapepoints3D_4], ignore_index=True, sort=False)
assignmentsDF

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,square flight,/veroviz/models/drone.gltf,100,75,0.000000,42.8,-78.800000,0,14.666667,...,220,orange,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
1,1,square flight,/veroviz/models/drone.gltf,100,75,14.666667,42.8,-78.800000,220,137.374072,...,220,orange,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
2,1,square flight,/veroviz/models/drone.gltf,100,75,137.374072,42.8,-78.770000,220,159.374072,...,0,orange,3,solid,0.8,True,Cesium.Color.RED,3,solid,0.8
3,2,straight flight,/veroviz/models/drone.gltf,100,75,0.000000,42.8,-78.800000,0,123.195008,...,219,orange,3,solid,0.8,True,Cesium.Color.GREEN,3,solid,0.8
4,3,trapezoid a,/veroviz/models/drone.gltf,100,75,0.000000,42.8,-78.800000,0,22.000000,...,220,orange,3,solid,0.8,True,Cesium.Color.BLACK,3,solid,0.8
5,3,trapezoid a,/veroviz/models/drone.gltf,100,75,22.000000,42.8,-78.796993,220,113.356473,...,220,orange,3,solid,0.8,True,Cesium.Color.BLACK,3,solid,0.8
6,3,trapezoid a,/veroviz/models/drone.gltf,100,75,113.356473,42.8,-78.774658,220,157.356473,...,0,orange,3,solid,0.8,True,Cesium.Color.BLACK,3,solid,0.8
7,4,trapezoid b,/veroviz/models/drone.gltf,100,75,0.000000,42.8,-78.800000,0,245.414811,...,0,orange,3,solid,0.8,True,Cesium.Color.BLACK,3,dashed,0.8
8,4,trapezoid b,/veroviz/models/drone.gltf,100,75,245.414811,42.8,-78.770000,0,319.703048,...,1114.32,orange,3,solid,0.8,True,Cesium.Color.BLACK,3,dashed,0.8
9,5,triangle,/veroviz/models/drone.gltf,100,75,0.000000,42.8,-78.800000,0,62.331989,...,220,orange,3,solid,0.8,True,Cesium.Color.LIGHTPINK,3,solid,0.8


In [78]:
# Generate Cesium materials to view all flight profiles:
>>> vrv.createCesium(
...     assignments = assignmentsDF, 
...     cesiumDir   = os.environ['CESIUMDIR'],
...     problemDir  = '/examples/shapepoints3D')

Message: File selector was written to /Users/murray/cesium/examples/shapepoints3D/;examples;shapepoints3D.vrv ...
Message: Configs were written to /Users/murray/cesium/examples/shapepoints3D/config.js ...
Message: Assignments (.js) were written to /Users/murray/cesium/examples/shapepoints3D/displayPaths.js ...
Message: Assignments (.czml) were written to /Users/murray/cesium/examples/shapepoints3D/routes.czml ...


In [79]:
# Example 6 - This example includes all of the functional arguments.

shapepoints3D = vrv.getShapepoints3D(
    odID               = 1,
    objectID           = 'drone', 
    modelFile          = '/veroviz/models/drone.gltf', 
    startTimeSec       = 120.0, 
    startLoc           = [42.80, -78.90], 
    endLoc             = [42.85, -78.95], 
    takeoffSpeedMPS    = 25, 
    cruiseSpeedMPS     = 25, 
    landSpeedMPS       = 25, 
    cruiseAltMetersAGL = 120, 
    routeType          = 'trapezoidal', 
    climbRateMPS       = 6, 
    descentRateMPS     = 6, 
    earliestLandTime   = 500, 
    loiterPosition     = 'arrivalAtAlt', 
    leafletColor       = 'orange', 
    leafletWeight      = 3, 
    leafletStyle       = 'solid', 
    leafletOpacity     = 0.8, 
    useArrows          = True, 
    cesiumColor        = 'Cesium.Color.ORANGE', 
    cesiumWeight       = 3, 
    cesiumStyle        = 'solid', 
    cesiumOpacity      = 0.8)

vrv.createCesium(
    assignments = shapepoints3D, 
    cesiumDir   = os.environ['CESIUMDIR'],
    problemDir  = 'examples/shapepoints3D/ex6')

Message: File selector was written to /Users/murray/cesium/examples/shapepoints3D/ex6/;examples;shapepoints3D;ex6.vrv ...
Message: Configs were written to /Users/murray/cesium/examples/shapepoints3D/ex6/config.js ...
Message: Assignments (.js) were written to /Users/murray/cesium/examples/shapepoints3D/ex6/displayPaths.js ...
Message: Assignments (.czml) were written to /Users/murray/cesium/examples/shapepoints3D/ex6/routes.czml ...


### 3D Shapepoints Demo
This final example combines several VeRoViz functions.

In this example, a truck and a drone start at the same location (node 1).  The drone will travel to nodes 2 and 3; the truck will travel directly to node 3.  The drone cannot land at node 3 until the truck arrives.

In [80]:
# Define 3 nodes
nodes = vrv.createNodesFromLocs(
    locs             = [[42.80, -78.80], 
                        [42.83, -78.75], 
                        [42.80, -78.70]], 
    snapToRoad       = True, 
    dataProvider     = 'ORS-online',
    dataProviderArgs = {'APIkey': os.environ['ORSKEY']})
nodes

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,1,42.801336,-78.802702,0,None,None,glyphicon,info-sign,blue,1,pin,Cesium.Color.BLUE,1
1,2,42.830042,-78.749774,0,None,None,glyphicon,info-sign,blue,2,pin,Cesium.Color.BLUE,2
2,3,42.801278,-78.700006,0,None,None,glyphicon,info-sign,blue,3,pin,Cesium.Color.BLUE,3


In [81]:
# Find the truck's travel time from node 1 to node 3
[truckTimeSec, truckDistMeters] = vrv.getTimeDistScalar2D(
            startLoc         = list(nodes[nodes['id']==1][['lat', 'lon']].values[0]),
            endLoc           = list(nodes[nodes['id']==3][['lat', 'lon']].values[0]),
            routeType        = 'fastest',
            dataProvider     = 'ORS-online',
            dataProviderArgs = {'APIkey': os.environ['ORSKEY']})
truckTimeSec

833.21

In [82]:
# Get the truck shapepoints, using expected duration
truckShapepoints = vrv.getShapepoints2D(
        odID             = 1,
        objectID         = 'truck', 
        modelFile        = 'veroviz/models/ub_truck.gltf', 
        modelScale       = 100,
        modelMinPxSize   = 75,
        startTimeSec     = 0,
        startLoc         = list(nodes[nodes['id']==1][['lat', 'lon']].values[0]),
        endLoc           = list(nodes[nodes['id']==3][['lat', 'lon']].values[0]),
        expDurationSec   = truckTimeSec, 
        routeType        = 'fastest', 
        dataProvider     = 'ORS-online',
        dataProviderArgs = {'APIkey': os.environ['ORSKEY']}, 
        leafletColor     = 'blue', 
        cesiumColor      = 'Cesium.Color.BLUE')

# Make the truck stay idle when it arrives at node 3:
truckShapepoints = vrv.addStaticAssignment(
    initAssignments = truckShapepoints, 
    odID            = 1, 
    objectID        = 'truck', 
    modelFile       = 'veroviz/models/ub_truck.gltf', 
    modelScale      = 100,
    modelMinPxSize  = 75,
    loc             = list(nodes[nodes['id']==3][['lat', 'lon']].values[0]), 
    startTimeSec    = max(truckShapepoints['endTimeSec']),
    endTimeSec      = -1)

In [83]:
# Route the drone from 1 to 2:
droneShapepoints_1 = vrv.getShapepoints3D(
        odID               = 2,
        objectID           = 'drone', 
        modelFile          = '/veroviz/models/drone.gltf',
        startTimeSec       = 0,
        startLoc           = list(nodes[nodes['id']==1][['lat', 'lon']].values[0]),
        endLoc             = list(nodes[nodes['id']==2][['lat', 'lon']].values[0]),
        cruiseAltMetersAGL = 220,
        takeoffSpeedMPS    = 15,
        cruiseSpeedMPS     = 20,
        landSpeedMPS       = 10,
        climbRateMPS       = None,
        descentRateMPS     = None,    
        routeType          = 'square',
        earliestLandTime   = -1,      # Can land as soon as it arrives.
        cesiumColor        = 'Cesium.Color.ORANGE')

# Route the drone from 2 to 3, but wait for the truck before landing:
droneShapepoints_2 = vrv.getShapepoints3D(
        odID               = 2,
        objectID           = 'drone', 
        modelFile          = '/veroviz/models/drone.gltf',
        startTimeSec       = max(droneShapepoints_1['endTimeSec']),
        startLoc           = list(nodes[nodes['id']==2][['lat', 'lon']].values[0]),
        endLoc             = list(nodes[nodes['id']==3][['lat', 'lon']].values[0]),
        cruiseAltMetersAGL = 220,
        takeoffSpeedMPS    = 15,
        cruiseSpeedMPS     = 20,
        landSpeedMPS       = 10,
        climbRateMPS       = None,
        descentRateMPS     = None,    
        routeType          = 'square',
        earliestLandTime   = truckTimeSec,    # Must wait for truck to arrive.
        loiterPosition     = 'arrivalAtAlt',
        cesiumColor        = 'Cesium.Color.ORANGE')

In [84]:
# Initialize an empty "assignments" dataframe:
assignmentsDF = vrv.initDataframe('assignments')

# Append to the assignments dataframe.
# The assignments dataframe will hold all shapepoints for both vehicles.
assignmentsDF = pd.concat([truckShapepoints, droneShapepoints_1, droneShapepoints_2], 
                          ignore_index=True, sort=False)
assignmentsDF

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,truck,/veroviz/models/ub_truck.gltf,100,75,0,42.801336,-78.802702,0,0,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
1,1,truck,/veroviz/models/ub_truck.gltf,100,75,0,42.801336,-78.802702,0,2.47573,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
2,1,truck,/veroviz/models/ub_truck.gltf,100,75,2.47573,42.801209,-78.803107,0,4.62812,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
3,1,truck,/veroviz/models/ub_truck.gltf,100,75,4.62812,42.801465,-78.803267,0,9.22126,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
4,1,truck,/veroviz/models/ub_truck.gltf,100,75,9.22126,42.801217,-78.804011,0,17.061,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
5,1,truck,/veroviz/models/ub_truck.gltf,100,75,17.061,42.802007,-78.804901,0,18.4172,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
6,1,truck,/veroviz/models/ub_truck.gltf,100,75,18.4172,42.802176,-78.804975,0,20.1634,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
7,1,truck,/veroviz/models/ub_truck.gltf,100,75,20.1634,42.802402,-78.805022,0,28.3395,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
8,1,truck,/veroviz/models/ub_truck.gltf,100,75,28.3395,42.803472,-78.804978,0,36.4954,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
9,1,truck,/veroviz/models/ub_truck.gltf,100,75,36.4954,42.804539,-78.804920,0,36.6698,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8


In [85]:
# Look at the routes in Leaflet.
# We can't see timing or altitude changes here.
vrv.createLeaflet(nodes=nodes, arcs=assignmentsDF)

In [86]:
# Generate Cesium materials so we can watch the vehicle movements:
>>> vrv.createCesium(
...     assignments = assignmentsDF, 
...     cesiumDir   = os.environ['CESIUMDIR'],
...     problemDir  = '/examples/shapepoints3D/demo')

Message: File selector was written to /Users/murray/cesium/examples/shapepoints3D/demo/;examples;shapepoints3D;demo.vrv ...
Message: Configs were written to /Users/murray/cesium/examples/shapepoints3D/demo/config.js ...
Message: Assignments (.js) were written to /Users/murray/cesium/examples/shapepoints3D/demo/displayPaths.js ...
Message: Assignments (.czml) were written to /Users/murray/cesium/examples/shapepoints3D/demo/routes.czml ...


## getTimeDist2D Module

This module contains the following function:
- `getTimeDist2D()`

### `getTimeDist2D()`
Generates two dictionaries; one for distance, one for time.  This is for vehicles that travel only on the ground (2-dimensional movement).

- See https://veroviz.org/docs/veroviz.getTimeDist2D.html#veroviz.getTimeDist2D.getTimeDist2D for complete function documentation.

In [87]:
# Import veroviz and check if the version is up-to-date
>>> import veroviz as vrv
>>> vrv.checkVersion()

'Your current installed version of veroviz is 0.2.0. You are up-to-date with the latest available version.'

In [88]:
# Generate a "nodes" dataframe from a collection of locations:
>>> locs = [
...     [42.1538, -78.4253],
...     [42.3465, -78.6234],
...     [42.6343, -78.1146]]
>>> exampleNodes = vrv.createNodesFromLocs(locs=locs)

In [89]:
# Example 1 - Generate time and distance matrices that only 
# consider Euclidean travel (default), 
# and assume the speed is a constant at 15 m/s. 
# No data provider is required/used in this case.
[timeSec, distMeters] = vrv.getTimeDist2D(
     nodes        = exampleNodes,
     routeType    = 'euclidean2D',
     speedMPS     = 15)
print("Travel time from node 1 to 2 is %.2f seconds" % (timeSec[1, 2]))
print("Travel distance from node 1 to 2 is %.2f meters" % (distMeters[1, 2]))

Travel time from node 1 to 2 is 1795.59 seconds
Travel distance from node 1 to 2 is 26933.80 meters


In [90]:
>>> # Example 2 - Use a data provider to find the time and distance matrices
>>> # according to the 'fastest' travel time metric. 
>>> import os
>>> ORS_API_KEY = os.environ['ORSKEY']   
>>> # If you don't have environment variables configured, uncomment and try the following line:   
>>> # ORS_API_KEY = 'YOUR_ORS_KEY_GOES_HERE'
>>> [timeSec, distMeters] = vrv.getTimeDist2D(
...     nodes        = exampleNodes,
...     routeType    = 'fastest',
...     dataProvider = 'ORS-online',
...     dataProviderArgs = { 'APIkey' : ORS_API_KEY })
>>> print("Travel time from node 1 to 2 is %.2f seconds" % (timeSec[1, 2]))
>>> print("Travel distance from node 1 to 2 is %.2f meters" % (distMeters[1, 2]))

Travel time from node 1 to 2 is 4326.08 seconds
Travel distance from node 1 to 2 is 45440.25 meters


In [91]:
>>> # Example 3 - Sometimes you don't need the full square matrix of travel data.  
>>> # Here, we find the time/distance information associated with traveling 
>>> # from node 3 to all other nodes:
>>> [timeSec, distMeters] = vrv.getTimeDist2D(
...     nodes            = exampleNodes,
...     matrixType       = 'one2many',
...     fromNodeID       = 3,
...     toNodeID         = None,
...     routeType        = 'fastest',
...     dataProvider     = 'ORS-online',
...     dataProviderArgs = { 'APIkey' : ORS_API_KEY })
>>> [timeSec, distMeters]

[{(3, 1): 4906.26, (3, 2): 5266.07, (3, 3): 0.0},
 {(3, 1): 79412.62, (3, 2): 77391.46, (3, 3): 0.0}]

In [92]:
>>> # Example 4 - Similarly, we can find the time/distance information 
>>> # for travel from all nodes to node 3:
>>> [timeSec, distMeters] = vrv.getTimeDist2D(
...     nodes            = exampleNodes,
...     matrixType       = 'many2one',
...     fromNodeID       = None,
...     toNodeID         = 3,
...     routeType        = 'fastest',
...     dataProvider     = 'ORS-online',
...     dataProviderArgs = { 'APIkey' : ORS_API_KEY })
>>> [timeSec, distMeters]

[{(1, 3): 4908.58, (2, 3): 5266.07, (3, 3): 0.0},
 {(1, 3): 79407.15, (2, 3): 77391.46, (3, 3): 0.0}]

In [93]:
>>> # Example 5 - As with the `getShapepoints2D()` function, you can 
>>> # provide the speed of the vehicle to override the speed data from a data provider.  
>>> # Here, we set a constant vehicle speed of 35 MPH (and convert to meters per second).
>>> # We also specify that we want time and distance outputs to be in Hours and Miles, respectively.
>>> # This example includes all functional arguments:
>>> [timeHours, distMiles] = vrv.getTimeDist2D(
...     nodes            = exampleNodes, 
...     matrixType       = 'all2all',
...     fromNodeID       = None, 
...     toNodeID         = None, 
...     outputTimeUnits  = 'hours',
...     outputDistUnits  = 'miles',
...     routeType        = 'fastest',
...     speedMPS         = vrv.convertSpeed(35, 'miles', 'hour', 'meters', 'second'),   # (MPH to m/s)
...     dataProvider     = 'ORS-online',
...     dataProviderArgs = {
...         'APIkey': ORS_API_KEY})
>>> [timeHours, distMiles]

[{(1, 1): 0.0,
  (1, 2): 0.8067237856157538,
  (1, 3): 1.4097553744303344,
  (2, 1): 0.8067184595637253,
  (2, 2): 0.0,
  (2, 3): 1.3739698083864018,
  (3, 1): 1.4098524861123194,
  (3, 2): 1.3739698083864018,
  (3, 3): 0.0},
 {(1, 1): 0.0,
  (1, 2): 28.235332496551386,
  (1, 3): 49.341438105061705,
  (2, 1): 28.23514608473039,
  (2, 2): 0.0,
  (2, 3): 48.08894329352406,
  (3, 1): 49.344837013931176,
  (3, 2): 48.08894329352406,
  (3, 3): 0.0}]

## getTimeDist3D Module
This module contains the following function:
- `getTimeDist3D()`

### `getTimeDist3D()`
This function calculates travel time and distance for vehicles that travel in 3-dimensional space (e.g., drones).  The function returns three dictionaries; one for time, one for ground distance, and one for overall (3D) travel distance.

- See https://veroviz.org/docs/veroviz.getTimeDist3D.html#veroviz.getTimeDist3D.getTimeDist3D for complete function documentation.

In [94]:
# Import veroviz and check if the version is up-to-date
>>> import veroviz as vrv
>>> vrv.checkVersion()

'Your current installed version of veroviz is 0.2.0. You are up-to-date with the latest available version.'

In [95]:
# Generate a "nodes" dataframe from a collection of locations:
>>> locs = [
...     [42.1538, -78.4253],
...     [42.3465, -78.6234],
...     [42.6343, -78.1146]]
>>> exampleNodes = vrv.createNodesFromLocs(locs=locs)

In [96]:
>>> # Example 1 - Calculate 'all2all' travel matrices for a drone with a 
>>> # 'square' flight profile.  There are 3 nodes, so the matrices will be 3x3.  
>>> [totalTime, totalGroundDistance, totalFlightDistance] = vrv.getTimeDist3D(
...     nodes              = exampleNodes,
...     routeType          = 'square',
...     cruiseAltMetersAGL = 120,
...     takeoffSpeedMPS    = 5,
...     cruiseSpeedMPS     = 12,
...     landSpeedMPS       = 2, 
...     outputDistUnits    = 'meters', 
...     outputTimeUnits    = 'seconds')
>>> print("Travel time from node 2 to node 3 is %.2f seconds" % (totalTime[2, 3]))
>>> print("Ground distance from node 2 to node 3 is %.2f meters" % (totalGroundDistance[2, 3]))
>>> print("Total flight distance from node 2 to node 3 is %.2f meters" % (totalFlightDistance[2, 3]))

Travel time from node 2 to node 3 is 4471.27 seconds
Ground distance from node 2 to node 3 is 52647.18 meters
Total flight distance from node 2 to node 3 is 52887.18 meters


In [97]:
>>> # Example 2 - Calculate 'one2many' travel matrices for a drone with a 
>>> # 'trapezoidal' flight profile, starting from node 2.  All functional 
>>> # arguments are included in this example.
>>> [timeSec, groundDist, totalDist] = vrv.getTimeDist3D(
...     nodes              = exampleNodes,
...     matrixType         = 'one2many', 
...     fromNodeID         = 2, 
...     toNodeID           = None, 
...     takeoffSpeedMPS    = 5, 
...     cruiseSpeedMPS     = 12, 
...     landSpeedMPS       = 5, 
...     cruiseAltMetersAGL = 120, 
...     routeType          = 'trapezoidal', 
...     climbRateMPS       = 1, 
...     descentRateMPS     = 1, 
...     outputDistUnits    = 'meters', 
...     outputTimeUnits    = 'seconds')
>>> print("Travel time from node 2 to node 3 is %.2f seconds" % (timeSec[2, 3]))
>>> print("Ground distance from node 2 to node 3 is %.2f meters" % (groundDist[2, 3]))
>>> print("Total flight distance from node 2 to node 3 is %.2f meters" % (totalDist[2, 3]))

Travel time from node 2 to node 3 is 4529.29 seconds
Ground distance from node 2 to node 3 is 52647.18 meters
Total flight distance from node 2 to node 3 is 52671.44 meters


---
## getTimeDistScalar2D Module
This module contains the following function:
- `getTimeDistScalar2D()`

### `getTimeDistScalar2D()`
Returns scalar values of time and distance between 2 points in 2 dimensions.

- See https://veroviz.org/docs/veroviz.getTimeDistScalar2D.html#veroviz.getTimeDistScalar2D.getTimeDistScalar2D for complete function documentation.

In [98]:
# Import veroviz and check if the version is up-to-date
>>> import veroviz as vrv
>>> vrv.checkVersion()

'Your current installed version of veroviz is 0.2.0. You are up-to-date with the latest available version.'

In [99]:
# Example 1 
>>> import os
>>>
>>> [timeSec, distMeters] = vrv.getTimeDistScalar2D(
...     startLoc         = [42.80, -78.80],
...     endLoc           = [42.90, -78.80],
...     routeType        = 'fastest',
...     dataProvider     = 'ORS-online',
...     dataProviderArgs = {'APIkey': os.environ['ORSKEY']})
>>> print("Travel time in seconds: %.2f" % (timeSec))
>>> print("Distance in meters: %.2f" % (distMeters))

Travel time in seconds: 1134.74
Distance in meters: 13891.61


In [100]:
# Example 2 - This example includes all functional arguments:
>>> [timeMin, distMiles] = vrv.getTimeDistScalar2D(
...     startLoc        = [42.80, -78.80], 
...     endLoc          = [42.90, -78.80],
...     outputTimeUnits = 'minutes',
...     outputDistUnits = 'miles',
...     routeType       = 'fastest',
...     speedMPS        = None,
...     dataProvider    = 'ORS-online',
...     dataProviderArgs = {'APIkey': os.environ['ORSKEY']})
>>> [timeMin, distMiles]

[18.912333333333333, 8.631867722171823]

---
## getTimeDistScalar3D Module
This module contains the following function:
- `getTimeDistScalar3D()`

### `getTimeDistScalar3D()`
Returns scalar values of time, ground distance, and total distance between 2 points in 3 dimensions.

- See https://veroviz.org/docs/veroviz.getTimeDistScalar3D.html#veroviz.getTimeDistScalar3D.getTimeDistScalar3D for complete function documentation.

In [101]:
>>> import veroviz as vrv
>>> vrv.checkVersion()

'Your current installed version of veroviz is 0.2.0. You are up-to-date with the latest available version.'

In [102]:
# Example 1 
>>> [timeSec, grndDistMeters, totalDistMeters] = vrv.getTimeDistScalar3D(
...     startLoc           = [42.80, -78.90],
...     endLoc             = [42.85, -78.95],
...     cruiseAltMetersAGL = 120,
...     takeoffSpeedMPS    = 15,
...     cruiseSpeedMPS     = 20,
...     landSpeedMPS       = 10,
...     routeType          = 'square')
>>> [timeSec, grndDistMeters, totalDistMeters]

[364.85047765232866, 6897.009553046573, 7137.009553046573]

In [103]:
# Example 2 - This example includes all functional arguments:
>>> [timeMin, grndDistMiles, totalDistMiles] = vrv.getTimeDistScalar3D(
...     startLoc           = [42.80, -78.90],
...     endLoc             = [42.85, -78.95],
...     routeType          = 'trapezoidal',
...     cruiseAltMetersAGL = 120,
...     takeoffSpeedMPS    = 25,
...     climbRateMPS       = 6,
...     cruiseSpeedMPS     = 25,
...     landSpeedMPS       = 25,
...     descentRateMPS     = 6,
...     outputTimeUnits    = 'minutes',
...     outputDistUnits    = 'miles')
>>> [timeMin, grndDistMiles, totalDistMiles]

[4.617491177183978, 4.28561370067641, 4.303774693834719]

---
## snapNodesToRoad Module
This module contains the following functions:
- `snapNodesToRoad()`
- `getSnapLoc()`
- `getSnapLocBatch()`

### `snapNodesToRoad()`
Updates the locations of nodes within a dataframe, such that each node becomes located on a road network.

- See https://veroviz.org/docs/veroviz.snapNodesToRoad.html#veroviz.snapNodesToRoad.snapNodesToRoad for complete function documentation.

In [104]:
# Import veroviz and check if the version is up-to-date:
>>> import veroviz as vrv
>>> vrv.checkVersion()

'Your current installed version of veroviz is 0.2.0. You are up-to-date with the latest available version.'

In [105]:
# Also, import os so we can use environment variables for data provider API keys:
>>> import os

In [106]:
# Create a nodes dataframe with 2 nodes that happen to be off of the road network:
>>> exampleNodes = vrv.createNodesFromLocs(
...     locs = [[42.80, -78.00],
...             [42.81, -78.004]],
...     leafletColor = 'red')

These nodes are not aligned with the road network.  The following examples demonstrate how to move the nodes to lat/lon locations that correspond to roads.

In [107]:
>>> # Use OSRM as data provider, and change node colors to green:
>>> snappedNodesOSRM = vrv.snapNodesToRoad(
...     nodes        = exampleNodes,
...     dataProvider = 'OSRM-online')
>>> snappedNodesOSRM['leafletColor'] = 'green'

In [108]:
# Display the original (red) and OSRM-snapped (green) nodes:
>>> myMap = vrv.createLeaflet(nodes=exampleNodes, mapBackground='Stamen Toner')
>>> myMap = vrv.createLeaflet(nodes=snappedNodesOSRM, mapObject=myMap)
>>> myMap

In [109]:
>>> # Use MapQuest as data provider, and change node colors to purple:
>>> snappedNodesMQ = vrv.snapNodesToRoad(
...     nodes            = exampleNodes,
...     dataProvider     = 'MapQuest',
...     dataProviderArgs = { 'APIkey' : os.environ['MAPQUESTKEY']})
>>> snappedNodesMQ['leafletColor'] = 'purple'

In [110]:
>>> # Add the MapQuest-snapped (purple) nodes to our map:
>>> myMap = vrv.createLeaflet(nodes=snappedNodesMQ, mapObject=myMap)
>>> myMap

In [111]:
>>> # Use ORS as data provider, and change node colors to blue:
>>> snappedNodesORS = vrv.snapNodesToRoad(
...     nodes            = exampleNodes,
...     dataProvider     = 'ORS-online',
...     dataProviderArgs = {'APIkey': os.environ['ORSKEY']})
>>> snappedNodesORS['leafletColor'] = 'blue'

In [112]:
>>> # Add the ORS-snapped (blue) nodes to our map:
>>> myMap = vrv.createLeaflet(nodes=snappedNodesORS, mapObject=myMap)
>>> myMap

### `getSnapLoc()`
Snap a given single location, given by [lat, lon] coordinates, to the nearest location on a road network.

- See https://veroviz.org/docs/veroviz.snapNodesToRoad.html#veroviz.snapNodesToRoad.getSnapLoc for complete function documentation.

In [113]:
# Import veroviz and check if the version is up-to-date:
>>> import veroviz as vrv
>>> vrv.checkVersion()

'Your current installed version of veroviz is 0.2.0. You are up-to-date with the latest available version.'

In [114]:
# Snap a given location to the nearest street:
>>> loc = [42.00, -78.00]
>>> snapLoc = vrv.getSnapLoc(
...     loc          = loc,
...     dataProvider = 'OSRM-online')
>>> snapLoc

[41.999401, -78.003876]

In [115]:
# View the original (red) and snapped (green) locations on a map
>>> myMap = vrv.addLeafletMarker(center=loc, fillColor='red')
>>> myMap = vrv.addLeafletMarker(center=snapLoc, fillColor='green', mapObject=myMap)
>>> myMap

NOTE: This function returns only latitude and longitude.  Any altitude values provided as inputs in the `loc` parameter will be lost in the return.

In [116]:
# Snap a given location, with altitude, to the nearest street:
>>> loc3D = [42.00, -78.00, 100]
>>> snapLoc = vrv.getSnapLoc(
...     loc          = loc3D,
...     dataProvider = 'OSRM-online')
>>> snapLoc

[41.999401, -78.003876, 100]

### `getSnapLocBatch()`
Snap multiple locations, each given by [lat, lon] coordinates, to their nearest locations on a road network.

- See https://veroviz.org/docs/veroviz.snapNodesToRoad.html#veroviz.snapNodesToRoad.getSnapLocBatch for complete function documentation.

In [117]:
# Import veroviz and check if the version is up-to-date:
>>> import veroviz as vrv
>>> vrv.checkVersion()

'Your current installed version of veroviz is 0.2.0. You are up-to-date with the latest available version.'

In [118]:
# Also, import os so we can use environment variables for data provider API keys:
>>> import os

In [119]:
# Snap a given list of location to their nearest streets:
>>> locs = [
...     [42.00, -78.00],
...     [42.10, -78.00],
...     [42.20, -78.00]]
>>> snapLocs = vrv.getSnapLocBatch(
...     locs             = locs,
...     dataProvider     = 'MapQuest',
...     dataProviderArgs = {'APIkey': os.environ['MAPQUESTKEY']})
>>> snapLocs

[[41.998485, -77.998512], [42.100039, -77.995549], [42.199122, -78.000786]]

In [120]:
>>> locs3D = [
...     [42.00, -78.00, 200],
...     [42.10, -78.00, 300],
...     [42.20, -78.00, 400]]
>>> snapLocs = vrv.getSnapLocBatch(
...     locs             = locs3D,
...     dataProvider     = 'MapQuest',
...     dataProviderArgs = {'APIkey': os.environ['MAPQUESTKEY']})
>>> snapLocs

[[41.998485, -77.998512, 200],
 [42.100039, -77.995549, 300],
 [42.199122, -78.000786, 400]]

---
## utilities Module
This module contains the following functions:

- Unit Conversions
    - `convertArea()`
    - `convertDistance()`
    - `convertSpeed()`
    - `convertTime()`
    
- Geometry
    - `distance2D()`
    - `distance3D()`
    - `distancePath2D()`
    - `findLocsAtTime()`
    - `geocode()`
    - `reverseGeocode()`
    - `getHeading()`
    - `getMapBoundary()`
    - `getConvexHull()`
    - `isPointInPoly()`
    - `isPathInPoly()`
    - `isPathCrossPoly()`
    - `isPassPath()`
    - `minDistLoc2Path()`
    - `pointInDistance()`

- Dataframes
    - `initDataframe()`
    - `convertMatricesDataframeToDictionary()` and `convertMatricesDictionaryToDataframe()`

- File Import/Export
    - `exportDataToCSV()` and `importDataFromCSV()`
    - `exportDataframe()` and `importDataframe()`

### `convertArea()`
Convert an area to different units.

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.convertArea for complete function documentation.

In [121]:
>>> import veroviz as vrv
>>> areaSQKM = 1.0
>>> areaSqMiles = vrv.convertArea(50, 'sqkm', 'sqmi')
>>> areaSqMiles

19.305

### `convertDistance()`
Convert a distance to different units.

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.convertDistance for complete function documentation.

In [122]:
>>> import veroviz as vrv
>>> distanceMiles = 1.0
>>> distanceKilometers = vrv.convertDistance(distanceMiles, 'miles', 'km')
>>> distanceKilometers

1.60934

### `convertSpeed()`
Convert a speed to different units.

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.convertSpeed for complete function documentation.

In [123]:
>>> import veroviz as vrv
>>> speedFPS = 10
>>> speedMPH = vrv.convertSpeed(speedFPS, 'ft', 's', 'mi', 'h')
>>> speedMPH

6.818198764711

### `convertTime()`
Convert a time to different units.

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.convertTime for complete function documentation.

In [124]:
>>> import veroviz as vrv
>>> timeHours = 1.5
>>> timeMinutes = vrv.convertTime(timeHours, 'h', 'min')
>>> timeMinutes

90.0

### `distance2D()`
Calculates the geodesic distance between two locations, using the geopy library.  Altitude is ignored.

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.distance2D for complete function documentation.

In [125]:
>>> import veroviz as vrv
>>> loc1 = [42.80, -78.90]
>>> loc2 = [42.82, -78.92]
>>> dist2D = vrv.distance2D(loc1, loc2)
>>> dist2D

2759.0335974131926

### `distance3D()`
Estimates the distance between two point, including changes in altitude.  The calculation combines geopy's geodesic distance (along the surface of an ellipsoidal model of the earth) with a simple estimate of additional travel distance due to altitude changes.

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.distance3D for complete function documentation.

In [126]:
>>> import veroviz as vrv
>>> loc1 = [42.80, -78.90, 0]
>>> loc2 = [42.82, -78.92, 300]
>>> dist3D = vrv.distance3D(loc1, loc2)
>>> dist3D

2775.2957304861734

###  `distancePath2D()`
Calculate the total geodesic distance along a path defined by [lat, lon] coordinates.  

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.distancePath2D for complete function documentation.

In [127]:
>>> import veroviz as vrv
>>> locs = [[42.80, -78.90], [42.82, -78.92], [42.84, -78.94]]
>>> path = vrv.distancePath2D(locs)
>>> path

5517.760959358346

###  `findLocsAtTime()`
Finds the estimated location of each unique `objectID` in an input `assignments` dataframe at the given time.  The output is a dictionary, where the keys are unique objectIDs.  The corresponding value for each `objectID` key will be `None` if the object is not defined at the given value of `timeSec`, the value will be a list of the form [lat, lon, alt] if a single match is found, or the value will be a list of lists of the form [[lat1, lon1, alt1], ..., [latn, lonn, altn]] if n matches are found.  In the latter case, this is typically indicative of duplicate entries in the assignments dataframe (as each object should not appear in multiple locations simultaneously).

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.findLocsAtTime for complete function documentation.

In [1]:
# Import veroviz and check if it's the latest version:
import veroviz as vrv
vrv.checkVersion()

'Your current installed version of veroviz is 0.2.2. You are up-to-date with the latest available version.'

In [2]:
# Define 5 node locations, as [lat, lon] pairs:
locs = [[42.8871085, -78.8731949],
        [42.8888311, -78.8649649],
        [42.8802158, -78.8660787],
        [42.8845705, -78.8762794],
        [42.8908031, -78.8770140]]

In [ ]:
# Generate a nodes dataframe from these locations:
myNodes = vrv.createNodesFromLocs(locs=locs)

In [ ]:
# Construct an assignments dataframe for two vehicles, a drone and a truck.
# The truck will visit nodes 1 -> 2 -> 3 -> 1.
# The drone will visit nodes 1 -> 4 -> 5 -> 1.
mySolution = {
    'truck': [[1,2], [2,3], [3,1]],
    'drone': [[1,4], [4,5], [5,1]]
}

In [ ]:
# Define some information about our 2 vehicles, for use below:
vehicleProperties = {
    'drone': {'model': 'veroviz/models/drone.gltf',
              'leafletColor': 'red',
              'cesiumColor': 'Cesium.Color.RED'},
    'truck': {'model': 'veroviz/models/ub_truck.gltf',
              'leafletColor': 'blue',
              'cesiumColor': 'Cesium.Color.BLUE'}
}

In [ ]:
# This example assumes the use of ORS as the data provider. 
# If you have saved your API key as an environment variable, you may use `os.environ` to access it:
import os
ORS_API_KEY = os.environ['ORSKEY']
# Otherwise, you may specify your key here:
# ORS_API_KEY = 'YOUR_ORS_KEY_GOES_HERE'

In [ ]:
# Initialize an empty assignments dataframe:
myAssignments = vrv.initDataframe('assignments')

In [ ]:
# Build assignments for the truck route:
endTimeSec = 0.0
for arc in mySolution['truck']:
    [myAssignments, endTimeSec] = vrv.addAssignment2D(
            initAssignments  = myAssignments,
            objectID         = 'truck',
            modelFile        = vehicleProperties['truck']['model'],
            startLoc         = list(myNodes[myNodes['id'] == arc[0]][['lat', 'lon']].values[0]),
            endLoc           = list(myNodes[myNodes['id'] == arc[1]][['lat', 'lon']].values[0]),
            startTimeSec     = endTimeSec,
            leafletColor     = vehicleProperties['truck']['leafletColor'],
            cesiumColor      = vehicleProperties['truck']['cesiumColor'], 
            routeType        = 'fastest',
            dataProvider     = 'ORS-online', 
            dataProviderArgs = {'APIkey': ORS_API_KEY})
myAssignments

In [ ]:
# Build assignments for the drone deliveries:
endTimeSec = 0.0
for arc in mySolution['drone']:
    [myAssignments, endTimeSec] = vrv.addAssignment3D(
        initAssignments    = myAssignments,
        objectID           = 'drone',
        modelFile          = vehicleProperties['drone']['model'],
        startLoc           = list(myNodes[myNodes['id'] == arc[0]][['lat', 'lon']].values[0]),
        endLoc             = list(myNodes[myNodes['id'] == arc[1]][['lat', 'lon']].values[0]),
        startTimeSec       = endTimeSec,
        takeoffSpeedMPS    = vrv.convertSpeed(30, 'miles', 'hr', 'meters', 'sec'),
        cruiseSpeedMPS     = vrv.convertSpeed(80, 'miles', 'hr', 'meters', 'sec'),
        landSpeedMPS       = vrv.convertSpeed( 5, 'miles', 'hr', 'meters', 'sec'),
        cruiseAltMetersAGL = vrv.convertDistance(350, 'feet', 'meters'),
        routeType          = 'square',
        leafletColor       = vehicleProperties['drone']['leafletColor'],
        cesiumColor        = vehicleProperties['drone']['cesiumColor'])
myAssignments

In [ ]:
# Show the nodes and assignments on a map:
vrv.createLeaflet(nodes=myNodes, arcs=myAssignments)

In [ ]:
# Find the location of each vehicle at time 30.0:
currentLocs = vrv.findLocsAtTime(assignments=myAssignments, timeSec=30.0)

# Or, we can just find the location of the drone at time 30.0:
# currentLocs = vrv.findLocsAtTime(
#    assignments=myAssignments[myAssignments['objectID'] == 'drone'], 
#    timeSec=30.0)

currentLocs

In [ ]:
# Display the estimated locations on a map:
myMap = vrv.createLeaflet(nodes=myNodes, arcs=myAssignments)
for objectID in currentLocs:
    if (type(currentLocs[objectID]) is list):
        # This objectID has at least 1 location at this time:
        if (type(currentLocs[objectID][0]) is list):
            # There were multiple matches for this objectID:
            for i in currentLocs[objectID]:
                myMap = vrv.addLeafletMarker(mapObject=myMap, center=i)
        else:
            # We only have one location for this objectID:
            myMap = vrv.addLeafletMarker(mapObject=myMap, 
                                         center=currentLocs[objectID], 
                                         radius=9, 
                                         fillOpacity=0.7, 
                                         fillColor='black')
myMap

###  `geocode()`
Convert a street address, city, state, or zip code to GPS coordinates ([lat, lon] format).

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.geocode for complete function documentation.

In [ ]:
# Import veroviz and check if the version is up-to-date:
import veroviz as vrv
vrv.checkVersion()

In [22]:
# The following examples assume the use of ORS or MapQuest as the data provider.  
# If you have saved your API keys as environment variables, you may use `os.environ` to access them:
import os
ORS_API_KEY = os.environ['ORSKEY']
MQ_API_KEY = os.environ['MAPQUESTKEY']

# Otherwise, you may specify your keys here:
# ORS_API_KEY = 'YOUR_ORS_KEY_GOES_HERE'
# MQ_API_KEY = 'YOUR_MAPQUEST_KEY_GOES_HERE'

In [19]:
# Example 1 - Find [lat, lon] of Buckingham Palace, without specifying a data provider:
myLoc = vrv.geocode(location='Westminster, London SW1A 1AA, United Kingdom')
myLoc

[51.5008719, -0.1252387]

In [20]:
# Example 2 - Find [lat, lon] of Buckingham Palace, using ORS-online as the data provider:
myLoc = vrv.geocode(location         ='Westminster, London SW1A 1AA, United Kingdom', 
                    dataProvider     ='ors-online', 
                    dataProviderArgs = {'APIkey': ORS_API_KEY})
myLoc

[51.497991, -0.12875]

In [23]:
# Example 3 - Find [lat, lon] of Seattle, Washington, USA:
myLoc = vrv.geocode(location         ='seattle, wa', 
                    dataProvider     ='mapquest', 
                    dataProviderArgs = {'APIkey': MQ_API_KEY})
myLoc

[47.603229, -122.33028]

In [24]:
# Example 4 - Find [lat, lon] of the state of Florida, USA:
myLoc = vrv.geocode(location         ='florida', 
                    dataProvider     ='ors-ONLINE', 
                    dataProviderArgs = {'APIkey': ORS_API_KEY})
myLoc

[27.97762, -81.769611]

In [25]:
# Example 5 - Find [lat, lon] of the Space Needle (in Seattle, WA):
myLoc = vrv.geocode(location         ='space needle', 
                    dataProvider     ='ors-ONLINE', 
                    dataProviderArgs = {'APIkey': ORS_API_KEY})
myLoc

[47.620336, -122.349314]

In [26]:
# Draw the geocoded location as a red dot on a Leaflet map:
vrv.addLeafletMarker(center=myLoc)

###  `reverseGeocode()`
Convert a GPS coordinate (of the form [lat, lon] or [lat, lon, alt]) to an address.  If altitude is provided it will be ignored.

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.reverseGeocode for complete function documentation.

In [ ]:
# The following examples assume the use of ORS or MapQuest as the data provider.  
# If you have saved your API keys as environment variables, you may use `os.environ` to access them:
import os
ORS_API_KEY = os.environ['ORSKEY']
MQ_API_KEY = os.environ['MAPQUESTKEY']

# Otherwise, you may specify your keys here:
# ORS_API_KEY = 'YOUR_ORS_KEY_GOES_HERE'
# MQ_API_KEY = 'YOUR_MAPQUEST_KEY_GOES_HERE'

In [27]:
# Example 1 -- Without specifying a dataProvider:
[loc, addr] = vrv.reverseGeocode(location=[47.603229, -122.33028])
loc

[47.6030474, -122.3302567]

In [28]:
addr

{'place_id': 18472401,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'node',
 'osm_id': 1769027877,
 'lat': '47.6030474',
 'lon': '-122.3302567',
 'display_name': 'SDOT, 4th Avenue, West Edge, International District/Chinatown, Seattle, King County, Washington, 98104, United States',
 'address': {'bicycle_parking': 'SDOT',
  'road': '4th Avenue',
  'neighbourhood': 'West Edge',
  'suburb': 'International District/Chinatown',
  'city': 'Seattle',
  'county': 'King County',
  'state': 'Washington',
  'postcode': '98104',
  'country': 'United States',
  'country_code': 'us'},
 'boundingbox': ['47.6029474', '47.6031474', '-122.3303567', '-122.3301567']}

In [29]:
# Example 2 -- Using MapQuest:
[loc, addr] = vrv.reverseGeocode(location         = [47.603229, -122.33028], 
                                 dataProvider     = 'MapQuest', 
                                 dataProviderArgs = {'APIkey': MQ_API_KEY})
loc

[47.603229, -122.33028]

In [30]:
addr

{'street': '431 James St',
 'adminArea6': '',
 'adminArea6Type': 'Neighborhood',
 'adminArea5': 'Seattle',
 'adminArea5Type': 'City',
 'adminArea4': 'King',
 'adminArea4Type': 'County',
 'adminArea3': 'WA',
 'adminArea3Type': 'State',
 'adminArea1': 'US',
 'adminArea1Type': 'Country',
 'postalCode': '98104',
 'geocodeQualityCode': 'L1AAA',
 'geocodeQuality': 'ADDRESS',
 'dragPoint': False,
 'sideOfStreet': 'R',
 'linkId': '0',
 'unknownInput': '',
 'type': 's',
 'latLng': {'lat': 47.603229, 'lng': -122.33028},
 'displayLatLng': {'lat': 47.603229, 'lng': -122.33028},
 'nearestIntersection': {'streetDisplayName': '4th Ave',
  'distanceMeters': '0.0',
  'latLng': {'longitude': -122.33028, 'latitude': 47.603229},
  'label': 'James St & 4th Ave'},
 'roadMetadata': {'speedLimitUnits': 'mph',
  'tollRoad': None,
  'speedLimit': 25}}

In [31]:
# Example 3 -- Using OpenRouteService:
[loc, addr] = vrv.reverseGeocode(location         = [47.603229, -122.33028], 
                                 dataProvider     = 'ORS-online', 
                                 dataProviderArgs = {'APIkey': ORS_API_KEY})
loc

[47.603077, -122.330139]

In [32]:
addr

{'id': 'node/4491511984',
 'gid': 'openstreetmap:venue:node/4491511984',
 'layer': 'venue',
 'source': 'openstreetmap',
 'source_id': 'node/4491511984',
 'name': '4th Ave & James St',
 'confidence': 0.8,
 'distance': 0.02,
 'accuracy': 'point',
 'country': 'United States',
 'country_gid': 'whosonfirst:country:85633793',
 'country_a': 'USA',
 'region': 'Washington',
 'region_gid': 'whosonfirst:region:85688623',
 'region_a': 'WA',
 'county': 'King County',
 'county_gid': 'whosonfirst:county:102086191',
 'county_a': 'KN',
 'locality': 'Seattle',
 'locality_gid': 'whosonfirst:locality:101730401',
 'neighbourhood': 'Pioneer Square',
 'neighbourhood_gid': 'whosonfirst:neighbourhood:85866047',
 'continent': 'North America',
 'continent_gid': 'whosonfirst:continent:102191575',
 'label': '4th Ave & James St, Seattle, WA, USA'}

### `getHeading()`
Finds the heading required to travel from a current location to a goal location.  North is 0-degrees, east is 90-degrees, south is 180-degrees, west is 270-degrees.

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.getHeading for complete function documentation.

In [128]:
>>> import veroviz as vrv
>>> locCurrent = [42.80, -78.90]
>>> locGoal    = [42.85, -78.85]
>>> heading = vrv.getHeading(locCurrent, locGoal)
>>> heading

36.24057197338239

In [129]:
>>> # View the arc from the current location to the goal:
>>> arc = vrv.createArcsFromLocSeq(locSeq = [locCurrent, locGoal])
>>> vrv.createLeaflet(arcs=arc)

### `getMapBoundary()`
Find the smallest rectangle that encloses a collection of nodes, arcs, assignments, and/or locations.  This function returns a list of lists, of the form [minLat, maxLon], [maxLat, minLon]].  This is equivalent to finding the southeast and northwest corners of the rectangle.

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.getMapBoundary for complete function documentation.

In [130]:
>>> import veroviz as vrv
>>>
>>> # Create 3 nodes, with blue pin markers (default):
>>> myNodes = vrv.createNodesFromLocs(
...     locs = [[42.1343, -78.1234], 
...             [42.5323, -78.2534], 
...             [42.9812, -78.1353]])
>>> 
>>> # Create 1 arc, with orange arrows (default):
>>> myArc = vrv.createArcsFromLocSeq(locSeq = [[42.62, -78.20], 
...                                            [42.92, -78.30]])
>>> 
>>> # Define 2 locations, with altitude.  (We'll color these purple later):
>>> myLocs = [[42.03, -78.26, 100], [42.78, -78.25, 200]] 
>>>
>>> # Find the boundary of these objects:
>>> myBoundary = vrv.getMapBoundary(nodes = myNodes,
...                                 arcs  = myArc,
...                                 locs  = myLocs)
>>> myBoundary

[[42.03, -78.1234], [42.9812, -78.3]]

In [131]:
>>> # Initialize a map with nodes (blue) and an arc (orange):
>>> myMap = vrv.createLeaflet(nodes = myNodes, 
...                           arcs  = myArc)
>>>
>>> # Add red (default) circle markers for the locations:
>>> for i in range(0, len(myLocs)):
...    myMap = vrv.addLeafletMarker(mapObject = myMap, 
...                                 center    = myLocs[i])    
>>>
>>> # Convert myBoundary to a 4-point polygon:
>>> myBoundingRegion = [myBoundary[0], 
...                     [myBoundary[0][0], myBoundary[1][1]], 
...                     myBoundary[1], 
...                     [myBoundary[1][0], myBoundary[0][1]]]
>>>
>>> # Add the bounding region to the map:
>>> myMap = vrv.createLeaflet(mapObject      = myMap, 
...                           boundingRegion = myBoundingRegion)
>>> # Display the map:
>>> myMap

### `getConvexHull()`
Find the convex hull of a set of points.

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.getConvexHull for complete function documentation.

In [132]:
>>> # Find the convex hull of 5 locs that straddle the Prime Meridian:
>>> import veroviz as vrv
>>> locs = [[51.4865,  0.0008], 
...         [51.4777, -0.0002], 
...         [51.4801,  0.0029], 
...         [51.4726, -0.0161], 
...         [51.4752,  0.0158]]
>>> convexHull = vrv.getConvexHull(locs)
>>> convexHull

[[51.4726, -0.0161], [51.4865, 0.0008], [51.4752, 0.0158]]

In [133]:
>>> # Display the 5 locations and the convex hull on a map:
>>> myMap = None
>>> for loc in locs:
...     myMap = vrv.addLeafletMarker(mapObject=myMap, center=loc)
>>> myMap = vrv.addLeafletPolygon(mapObject=myMap, points=convexHull)
>>> myMap

### `isPointInPoly()`
Determine if a point is inside a polygon.  Points that are along the perimeter of the polygon (including vertices) are considered to be "inside".

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.isPointInPoly for complete function documentation.

In [134]:
>>> import veroviz as vrv

In [135]:
# Example 1 - Location is inside polygon:
>>> loc = [42.03, -78.05]
>>> poly = [[42.00, -78.00], [42.10, -78.10], [42.00, -78.10]]
>>> vrv.isPointInPoly(loc, poly)

True

In [136]:
>>> myMap = vrv.addLeafletMarker(center = loc)
>>> myMap = vrv.addLeafletPolygon(mapObject = myMap, points = poly)
>>> myMap

In [137]:
# Example 2 - Location is outside polygon:
>>> loc = [42.07, -78.05]
>>> poly = [[42.00, -78.00], [42.10, -78.10], [42.00, -78.10]]
>>> vrv.isPointInPoly(loc, poly)

False

In [138]:
>>> myMap = vrv.addLeafletMarker(center = loc)
>>> myMap = vrv.addLeafletPolygon(mapObject = myMap, points = poly)
>>> myMap

In [139]:
# Example 3 - Location is on the polygon boundary:
>>> loc = [42.05, -78.10]
>>> poly = [[42.00, -78.00], [42.10, -78.10], [42.00, -78.10]]
>>> vrv.isPointInPoly(loc, poly)

True

In [140]:
>>> myMap = vrv.addLeafletMarker(center = loc)
>>> myMap = vrv.addLeafletPolygon(mapObject = myMap, points = poly)
>>> myMap

In [141]:
# Example 4 - Location is on a polygon vertex:
>>> loc = [42.10, -78.10]
>>> poly = [[42.00, -78.00], [42.10, -78.10], [42.00, -78.10]]
>>> vrv.isPointInPoly(loc, poly)

True

In [142]:
>>> myMap = vrv.addLeafletMarker(center = loc)
>>> myMap = vrv.addLeafletPolygon(mapObject = myMap, points = poly)
>>> myMap

In [143]:
# Example 5 - Non-convex poly region:
>>> loc = [42.50, -78.90]
>>> poly = [[42.00, -78.00], [43.00, -78.00], [42.2, -78.5], [43.00, -79.00], [42.00, -79.00]]
>>> vrv.isPointInPoly(loc, poly)

True

In [144]:
>>> myMap = vrv.addLeafletMarker(center = loc)
>>> myMap = vrv.addLeafletPolygon(mapObject = myMap, points = poly)
>>> myMap

In [145]:
# Example 6 - Altitudes are included (but ignored):
>>> loc = [42.05, -78.10, 100]
>>> poly = [[42.00, -78.00, 200], [42.10, -78.10, 300], [42.00, -78.10, 200]]
>>> vrv.isPointInPoly(loc, poly)

True

In [146]:
>>> myMap = vrv.addLeafletMarker(center = loc)
>>> myMap = vrv.addLeafletPolygon(mapObject = myMap, points = poly)
>>> myMap

### `isPathInPoly()`
Determine if a given path is completely within the boundary of a polygon.  

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.isPathInPoly for complete function documentation.

In [147]:
>>> import veroviz as vrv

In [148]:
# Example 1 - Entire path is inside polygon:
>>> path = [[42.50, -78.10], [42.50, -78.50], [42.50, -78.90]]
>>> poly = [[42.00, -78.00], [43.00, -78.00], [43.00, -79.00], [42.00, -79.00]]
>>> vrv.isPathInPoly(path, poly)

True

In [149]:
>>> myMap = vrv.addLeafletPolyline(points = path)
>>> myMap = vrv.addLeafletPolygon(mapObject = myMap, points = poly)
>>> myMap

In [150]:
# Example 2 - One of the vertices is on the edge of the polygon:
>>> path = [[42.50, -78.10], [43.00, -78.50], [42.50, -78.90]]
>>> poly = [[42.00, -78.00], [43.00, -78.00], [43.00, -79.00], [42.00, -79.00]]
>>> vrv.isPathInPoly(path, poly)

False

In [151]:
>>> myMap = vrv.addLeafletPolyline(points = path)
>>> myMap = vrv.addLeafletPolygon(mapObject = myMap, points = poly)
>>> myMap

In [152]:
# Example 3 - Part of the path is outside of the polygon:
>>> path = [[42.50, -78.10], [43.10, -78.50], [42.50, -78.90]]
>>> poly = [[42.00, -78.00], [43.00, -78.00], [43.00, -79.00], [42.00, -79.00]]
>>> vrv.isPathInPoly(path, poly)

False

In [153]:
>>> myMap = vrv.addLeafletPolyline(points = path)
>>> myMap = vrv.addLeafletPolygon(mapObject = myMap, points = poly)
>>> myMap

In [154]:
# Example 4 - Endpoints are in the polygon, but the poly isn't convex:
>>> path = [[42.50, -78.10], [42.50, -78.90]]
>>> poly = [[42.00, -78.00], [43.00, -78.00], [42.2, -78.5], [43.00, -79.00], [42.00, -79.00]]
>>> vrv.isPathInPoly(path, poly)

False

In [155]:
>>> myMap = vrv.addLeafletPolyline(points = path)
>>> myMap = vrv.addLeafletPolygon(mapObject = myMap, points = poly)
>>> myMap

In [156]:
# Example 5 - Path and poly coordinates include altitude (which is ignored):
>>> path = [[42.50, -78.10, 100], [42.50, -78.90, 200]]
>>> poly = [[42.00, -78.00, 100], 
...         [43.00, -78.00, 100], 
...         [42.2, -78.5, 100], 
...         [43.00, -79.00, 200], 
...         [42.00, -79.00, 200]]
>>> vrv.isPathInPoly(path, poly)

False

In [157]:
>>> myMap = vrv.addLeafletPolyline(points = path)
>>> myMap = vrv.addLeafletPolygon(mapObject = myMap, points = poly)
>>> myMap

### `isPathCrossPoly()`

Determine if a given path crosses the boundary of a polygon.

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.isPathCrossPoly for complete function documentation.

In [158]:
>>> import veroviz as vrv

In [159]:
# Example 1 - Entire path is inside poly:
>>> path = [[42.50, -78.10], [42.50, -78.50], [42.50, -78.90]]
>>> poly = [[42.00, -78.00], [43.00, -78.00], [43.00, -79.00], [42.00, -79.00]]
>>> vrv.isPathCrossPoly(path, poly)

False

In [160]:
>>> myMap = vrv.addLeafletPolyline(points = path)
>>> myMap = vrv.addLeafletPolygon(mapObject = myMap, points = poly)
>>> myMap

In [161]:
# Example 2 - One of the vertices is on the edge of poly:
>>> path = [[42.50, -78.10], [43.00, -78.50], [42.50, -78.90]]
>>> poly = [[42.00, -78.00], [43.00, -78.00], [43.00, -79.00], [42.00, -79.00]]
>>> vrv.isPathCrossPoly(path, poly)

True

In [162]:
>>> myMap = vrv.addLeafletPolyline(points = path)
>>> myMap = vrv.addLeafletPolygon(mapObject = myMap, points = poly)
>>> myMap

In [163]:
# Example 3 - Part of the path is outside of poly:
>>> path = [[42.50, -78.10], [43.10, -78.50], [42.50, -78.90]]
>>> poly = [[42.00, -78.00], [43.00, -78.00], [43.00, -79.00], [42.00, -79.00]]
>>> vrv.isPathCrossPoly(path, poly)

True

In [164]:
>>> myMap = vrv.addLeafletPolyline(points = path)
>>> myMap = vrv.addLeafletPolygon(mapObject = myMap, points = poly)
>>> myMap

In [165]:
# Example 4 - Endpoints are in poly, but poly isn't convex:
>>> path = [[42.50, -78.10], [42.50, -78.90]]
>>> poly = [[42.00, -78.00], 
...         [43.00, -78.00], 
...         [42.2, -78.5], 
...         [43.00, -79.00], 
...         [42.00, -79.00]]
>>> vrv.isPathCrossPoly(path, poly)

True

In [166]:
# Example 5 - Path and poly include altitudes (which are ignored):
>>> path = [[42.50, -78.10, 100], [42.50, -78.90, 300]]
>>> poly = [[42.00, -78.00, 100], 
...         [43.00, -78.00, 200], 
...         [42.2, -78.5, 100], 
...         [43.00, -79.00, 300], 
...         [42.00, -79.00, 100]]
>>> vrv.isPathCrossPoly(path, poly)

True

### `isPassPath()`
Determine if any point along a path is within tolerance meters of a stationary point.  (Did our path pass by the target?)

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.isPassPath for complete function documentation.

In [167]:
>>> import veroviz as vrv
>>> path = [[42.50, -78.40], [42.50, -78.60], [42.40, -78.70]]

In [168]:
# Example 1 - The distance from the location to the path exceeds the tolerance.
>>> awayLoc = [42.51, -78.50]
>>> vrv.isPassPath(awayLoc, path, 1000)

False

In [169]:
# Find the minimum distance, in meters, from the location to the path:
vrv.minDistLoc2Path(awayLoc, path)

1105.9845259826711

In [170]:
>>> myMap = vrv.addLeafletMarker(center = awayLoc)
>>> myMap = vrv.addLeafletPolyline(mapObject = myMap, points = path)
>>> myMap

In [171]:
# Example 2 - The distance from the location to the path is within the tolerance.
>>> closeLoc = [42.505, -78.50]
>>> vrv.isPassPath(closeLoc, path, 1000)

True

In [172]:
# Find the minimum distance, in meters, from the location to the path:
vrv.minDistLoc2Path(closeLoc, path)

550.5689415111023

In [173]:
>>> myMap = vrv.addLeafletMarker(center = closeLoc)
>>> myMap = vrv.addLeafletPolyline(mapObject = myMap, points = path)
>>> myMap

In [174]:
# Example 3 - Location and path include altitudes (which are ignored):
>>> loc  = [42.505, -78.50, 100]
>>> path = [[42.50, -78.40, 100], 
...         [42.50, -78.60, 200], 
...         [42.40, -78.70, 100]]
>>> vrv.isPassPath(loc, path, 1000)

True

### `minDistLoc2Path()`
Calculate the minimum distance, in [meters], from a single stationary location (target) to any point along a path.

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.minDistLoc2Path for complete function documentation.

In [175]:
>>> import veroviz
>>> path = [[42.50, -78.40], [42.50, -78.60], [42.40, -78.70]]
>>> loc1 = [42.50, -78.50]
>>> loc2 = [42.51, -78.50]
>>> loc3 = [42.51, -78.3]

In [176]:
# Example 1 - The location is on the path:
>>> vrv.minDistLoc2Path(loc1, path)

0.0

In [177]:
# Example 2 - The minimum distance is between points on the path:
>>> vrv.minDistLoc2Path(loc2, path)

1105.9845259826711

In [178]:
# Example 3 - The minimum distance is to an endpoint of the path:
>>> vrv.minDistLoc2Path(loc3, path)

8293.970453010765

In [179]:
>>> myMap = vrv.addLeafletMarker(center=loc1, fillColor='blue')
>>> myMap = vrv.addLeafletMarker(mapObject=myMap, center=loc2, fillColor='green')
>>> myMap = vrv.addLeafletMarker(mapObject=myMap, center=loc3, fillColor='purple')
>>> myMap = vrv.addLeafletPolyline(mapObject=myMap, points=path)
>>> myMap

In [180]:
# Example 4 - The location and path include altitudes (which are ignored):
>>> path2 = [[42.50, -78.40, 100], 
...          [42.50, -78.60, 200], 
...          [42.40, -78.70, 100]]
>>> loc4  = [42.51, -78.3, 300]
>>> vrv.minDistLoc2Path(loc4, path2)

8293.970453010765

### `pointInDistance2D()`
Find the [lat, lon, alt] coordinate of a point that is a given distance away from a current location at a given heading.  This can be useful for determining where a vehicle may be in the future (assuming constant velocity and straight-line travel).

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.pointInDistance2D for complete function documentation.

In [181]:
>>> import veroviz as vrv
>>> startPt  = [42.80, -78.30, 200]
>>> heading  = 45 # degrees. travel northeast.
>>> distance = 300 # meters.
>>> 
>>> endPt = vrv.pointInDistance2D(startPt, heading, distance)
>>> endPt

[42.801909541371096, -78.2974067756287, 200]

In [182]:
myArc = vrv.createArcsFromLocSeq(locSeq = [startPt, endPt])
myMap = vrv.createLeaflet(arcs=myArc)
myMap = vrv.addLeafletMarker(mapObject=myMap, center=startPt, fillColor='red')
myMap = vrv.addLeafletMarker(mapObject=myMap, center=endPt, fillColor='green')
myMap

### Dataframes

### `initDataframe()`
Return an empty dataframe of a given type.  The options are 'Nodes', 'Arcs', and 'Assignments'.  These options are case insensitive.

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.initDataframe for complete function documentation.

In [78]:
>>> import veroviz as vrv
>>> newNodes = vrv.initDataframe('Nodes')
>>> newNodes

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText


### `convertMatricesDictionaryToDataframe()`
This function is intended for use with time/distance matrices, which are stored in veroviz as Python dictionaries. This function transforms a matrix dictionary into a pandas dataframe.  The dictionary is assumed to have 2-tuple indices (the first index represents the ID of the "from" location, the second index is the ID of the "to" location).  In the resulting pandas dataframe, the row indices will represent the "from" location, the column indices the "to" location.

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.convertMatricesDictionaryToDataframe for complete function documentation.

### `convertMatricesDataframeToDictionary()` 
This function is intended for use with time/distance matrices, which are stored in veroviz as Python dictionaries. This function transforms a matrix dataframe into  a dictionary, such that the indices of columns and rows become a tuple key for the dictionary.

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.convertMatricesDataframeToDictionary for complete function documentation.

In [80]:
>>> import veroviz as vrv
>>> locs = [
...     [42.1538, -78.4253], 
...     [42.3465, -78.6234], 
...     [42.6343, -78.1146]]
>>> exampleNodes = vrv.createNodesFromLocs(locs=locs)
>>> [timeDict, distDict] = vrv.getTimeDist2D(
...     nodes        = exampleNodes, 
...     routeType    = 'fastest', 
...     dataProvider = 'OSRM-online')
>>> [timeDict]

[{(1, 1): 0.0,
  (1, 2): 2531.0,
  (1, 3): 3625.3,
  (2, 1): 2483.2,
  (2, 2): 0.0,
  (2, 3): 3619.7,
  (3, 1): 3565.7,
  (3, 2): 3669.8,
  (3, 3): 0.0}]

In [81]:
>>> print("The travel time from node 1 to node 2 is %.2f seconds" % (timeDict[1, 2]))

The travel time from node 1 to node 2 is 2531.00 seconds


In [82]:
>>> # timeDict is a dictionary.  Convert to a dataframe:
>>> timeDF = vrv.convertMatricesDictionaryToDataframe(timeDict)
>>> timeDF

,1,2,3
1,0.0,2531.0,3625.3
2,2483.2,0.0,3619.7
3,3565.7,3669.8,0.0


In [83]:
>>> # NOTE:  The travel time from 1 to 2 is NOT found by timeDF[1][2].
>>> # INSTEAD, you must use timeDF[2][1]
>>> # Pandas uses the form timeDF[COLUMN_INDEX][ROW_INDEX]
>>> timeDF[1][2], timeDF[2][1], timeDict[1, 2], timeDict[2, 1]

(2483.2, 2531.0, 2531.0, 2483.2)

In [84]:
>>> timeDict2 = vrv.convertMatricesDataframeToDictionary(timeDF)
>>> timeDict2
>>> # This should be the same as `timeDict`

{(1, 1): 0.0,
 (1, 2): 2531.0,
 (1, 3): 3625.3,
 (2, 1): 2483.2,
 (2, 2): 0.0,
 (2, 3): 3619.7,
 (3, 1): 3565.7,
 (3, 2): 3669.8,
 (3, 3): 0.0}

In [85]:
>>> # Find the travel time *from* 1 *to* 3:
>>> timeDict2[1,3]

3625.3

### File Import/Export

### `exportDataToCSV()`  
Export a dataframe or python time/distance matrix dictionary to a `.csv` file.

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.exportDataToCSV for complete function documentation.

### `importDataFromCSV()`
Import from a `.csv` file into a dataframe or python time/distance matrix dictionary.

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.importDataFromCSV for complete function documentation.

In [86]:
# Import veroviz and check if it is the latest version:
>>> import veroviz as vrv
>>> vrv.checkVersion()

'Your current installed version of veroviz is 0.2.3, the latest version available is 0.2.2. To update to the latest version, type `pip install --upgrade veroviz` at a command-line prompt.'

In [87]:
# Create a nodes dataframe:
>>> nodesDF = vrv.createNodesFromLocs(
...              locs = [[42.1538, -78.4253], 
...                      [42.3465, -78.6234], 
...                      [42.6343, -78.1146]])
>>> nodesDF

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,1,42.1538,-78.4253,0,None,None,glyphicon,info-sign,blue,1,pin,Cesium.Color.BLUE,1
1,2,42.3465,-78.6234,0,None,None,glyphicon,info-sign,blue,2,pin,Cesium.Color.BLUE,2
2,3,42.6343,-78.1146,0,None,None,glyphicon,info-sign,blue,3,pin,Cesium.Color.BLUE,3


In [88]:
# Save the nodesDF dataframe as a .csv file in a subdirectory named "test":
>>> vrv.exportDataToCSV(nodesDF, 'test/nodes.csv')    

Message: Data written to test/nodes.csv.


In [89]:
# Import the dataframe we just saved:
>>> importedNodes = vrv.importDataFromCSV(
...     dataType = 'nodes',
...     filename = 'test/nodes.csv')
>>> importedNodes

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,1,42.1538,-78.4253,0,NaN,NaN,glyphicon,info-sign,blue,1,pin,Cesium.Color.BLUE,1
1,2,42.3465,-78.6234,0,NaN,NaN,glyphicon,info-sign,blue,2,pin,Cesium.Color.BLUE,2
2,3,42.6343,-78.1146,0,NaN,NaN,glyphicon,info-sign,blue,3,pin,Cesium.Color.BLUE,3


In [90]:
>>> # If the data type is inconsistent with the data, 
>>> # an error message will be thrown and nothing will be imported.
>>> importedArcs = vrv.importDataFromCSV(
...     dataType = 'arcs', 
...     filename = 'test/nodes.csv')

Error: odID is missing from arcs dataframe.  To see all required columns, call 'initDataframe('arcs')'.  test/nodes.csv was not successfully imported.


Similarly we can import and export an 'arcs' or 'assignments' dataframe.

For time/distance matrices, they are saved as dictionaries in VeRoViz.  Here is an example of how to import/export them.

In [98]:
# Get travel time/distance matrices using the nodes we just created:
>>> [timeDict, distDict] = vrv.getTimeDist2D(
...           nodes        = nodesDF, 
...           routeType    = 'fastest', 
...           dataProvider = 'OSRM-online')
>>> timeDict

{(1, 1): 0.0,
 (1, 2): 2531.0,
 (1, 3): 3625.3,
 (2, 1): 2483.2,
 (2, 2): 0.0,
 (2, 3): 3619.7,
 (3, 1): 3565.7,
 (3, 2): 3669.8,
 (3, 3): 0.0}

In [99]:
# Export the time dictionary to a .csv file in a subdirectory named "test":
>>> vrv.exportDataToCSV(data = timeDict, filename = 'test/timeMatrix.csv')

Message: Data written to test/timeMatrix.csv.


In [100]:
# Import the saved dictionary
>>> importedTime = vrv.importDataFromCSV(
...     dataType = 'matrix',
...     filename = 'test/timeMatrix.csv')
>>> importedTime

{(1, 1): 0.0,
 (1, 2): 2531.0,
 (1, 3): 3625.3,
 (2, 1): 2483.2,
 (2, 2): 0.0,
 (2, 3): 3619.7,
 (3, 1): 3565.7,
 (3, 2): 3669.8,
 (3, 3): 0.0}

### `exportDataframe()` 
Exports a nodes, arcs, or assignments dataframe to a `.csv` file.

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.exportDataframe for complete function documentation.

### `importDataframe()`
Imports a VeRoViz nodes, arcs, or assignments dataframe from a .csv file.  This function returns a pandas dataframe.

- See https://veroviz.org/docs/veroviz.utilities.html#veroviz.utilities.importDataframe for complete function documentation.

In [101]:
# Import veroviz and check if it is the latest version:
>>> import veroviz as vrv
>>> vrv.checkVersion()

'Your current installed version of veroviz is 0.2.3, the latest version available is 0.2.2. To update to the latest version, type `pip install --upgrade veroviz` at a command-line prompt.'

In [102]:
# Create a nodes dataframe:
>>> nodesDF = vrv.createNodesFromLocs(locs=[
...     [42.1538, -78.4253], 
...     [42.3465, -78.6234], 
...     [42.6343, -78.1146]])
>>> nodesDF

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,1,42.1538,-78.4253,0,None,None,glyphicon,info-sign,blue,1,pin,Cesium.Color.BLUE,1
1,2,42.3465,-78.6234,0,None,None,glyphicon,info-sign,blue,2,pin,Cesium.Color.BLUE,2
2,3,42.6343,-78.1146,0,None,None,glyphicon,info-sign,blue,3,pin,Cesium.Color.BLUE,3


In [103]:
# Save the nodesDF dataframe as a .csv file in a subdirectory named "test":
>>> vrv.exportDataframe(dataframe = nodesDF, filename = 'test/nodes.csv')

Message: Data written to test/nodes.csv.


In [104]:
# Import the saved dataframe:
>>> importedNodesDF = vrv.importDataframe('test/nodes.csv')
>>> importedNodesDF

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,1,42.1538,-78.4253,0,NaN,NaN,glyphicon,info-sign,blue,1,pin,Cesium.Color.BLUE,1
1,2,42.3465,-78.6234,0,NaN,NaN,glyphicon,info-sign,blue,2,pin,Cesium.Color.BLUE,2
2,3,42.6343,-78.1146,0,NaN,NaN,glyphicon,info-sign,blue,3,pin,Cesium.Color.BLUE,3
